In [1]:


# !pip install ultralytics





In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'
import ultralytics
from ultralytics import YOLO
import cv2 as cv
import time
# import fastreid.modeling.heads 
model = YOLO("yolov8n.pt")
model2=YOLO("yolo11n.pt")
model3=YOLO("yolov8m.pt")
model4=YOLO("yolov8x.pt")


# Your other imports like numpy, pandas, etc. go here

In [3]:
# !pip install opencv-python

In [4]:
# !pip install chardet

In [3]:
import cv2 as cv
import torch
import torchvision.transforms as T
from PIL import Image # <--- Import the Image class from Pillow

# --- 1. Load the image using OpenCV ---

def transform(img):
# --- 2. Define the Transformation Pipeline (no changes here) ---
    image_height = 256
    image_width = 128
    pixel_mean = [0.485, 0.456, 0.406]
    pixel_std = [0.229, 0.224, 0.225]

    transform = T.Compose([
        T.Resize((image_height, image_width)),
        T.ToTensor(),
        T.Normalize(mean=pixel_mean, std=pixel_std)
    ])

    # --- 3. Apply the Transformations ---

    # a. Convert color from BGR (OpenCV) to RGB
    rgb_crop = cv.cvtColor(img, cv.COLOR_BGR2RGB)


    # b. Convert the NumPy array to a PIL Image  <--- THIS IS THE FIX
    pil_image = Image.fromarray(rgb_crop)

    # c. Apply the preprocessing pipeline to the PIL Image
    tensor = transform(pil_image)

    # d. Add the batch dimension
    input_batch = tensor.unsqueeze(0)

    # e. Move to the correct device
    device = "cuda" if torch.cuda.is_available() else "cpu"
    input_batch = input_batch.to(device)
    
    
    return input_batch

In [4]:

import torch
import os
from pathlib import Path

# It is assumed that this script is run from a directory that contains
# the cloned 'fast-reid' repository.
# Add the fast-reid repository to the Python path.
import sys
sys.path.append('fast-reid')

from fastreid.config import get_cfg
from fastreid.modeling import build_model
from fastreid.utils.checkpoint import Checkpointer
import fastreid.modeling.heads 
import fastreid.modeling.backbones
def setup_cfg(config_file, weights_file):
    """
    Creates a FastReID config object and merges it with the specified
    configuration file and weights.
    """
    # 1. Get a copy of the default configuration
    cfg = get_cfg()

    # 2. Add project-specific configurations if any
    # (Not needed for this baseline model)

    # 3. Merge from the specified YAML configuration file
    # This file contains the model architecture and dataset specifics.
    cfg.merge_from_file(config_file)

    # 4. Set the path to the pre-trained model weights.
    # This is a critical step for loading the desired model state.
    cfg.MODEL.WEIGHTS = weights_file

    # 5. Set the device to use for inference.
    # Use 'cuda' for GPU or 'cpu' for CPU.
    cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    cfg.MODEL.HEADS.NUM_CLASSES = 751
    # 6. Freeze the configuration.
    # This makes the config object immutable and prevents accidental changes.
    cfg.freeze()
    
    return cfg

def main():
    """
    Main function to load the model and print its architecture.
    """
    # --- Configuration ---
    # Define paths relative to the project structure.
    project_root = Path('.')
    fastreid_repo_path = project_root / 'fast-reid'
    
    # Path to the configuration YAML file.
    config_file_path = fastreid_repo_path / 'configs/Market1501/bagtricks_R101-ibn.yml'
    
    # Path to the pre-trained weights file.
    weights_file_path = project_root / "C:/pooo/models/market_bot_R101-ibn.pth"

    # Verify that the necessary files exist.
    if not config_file_path.is_file():
        print(f"Error: Config file not found at {config_file_path}")
        return
    if not weights_file_path.is_file():
        print(f"Error: Weights file not found at {weights_file_path}")
        print("Please download 'market_bot_R50.pth' and place it in the 'models/' directory.")
        return

    # --- Model Loading ---
    # 1. Setup the configuration object.
    cfg = setup_cfg(str(config_file_path), str(weights_file_path))
    print(f"Configuration loaded successfully. Using device: {cfg.MODEL.DEVICE}")

    # 2. Build the model.
    # The `build_model` function reads the configuration and constructs the
    # corresponding model architecture. It also automatically loads the weights
    # specified in `cfg.MODEL.WEIGHTS`.
    model = build_model(cfg)
    
    print("Model built successfully.")

    # The Checkpointer is used internally by `build_model` but can also be
    # used explicitly if needed. For this use case, `build_model` handles it.
    # Checkpointer(model).load(cfg.MODEL.WEIGHTS) # This is done inside build_model

    # 3. Set the model to evaluation mode.
    # This is crucial for consistent results during inference as it disables
    # layers like Dropout and uses running statistics for Batch Normalization.
    model.eval()
    print("Model set to evaluation mode.")

    # --- Robust Weight Verification ---
   # --- Manual Weight Correction and Loading ---
    print("\n--- Manually Correcting and Loading Weights ---")
    
    device = torch.device(cfg.MODEL.DEVICE)
    weights_from_file = torch.load(weights_file_path, map_location=device)
    
    # Check if weights are nested under a 'model' key
    if 'model' in weights_from_file:
        weights_from_file = weights_from_file['model']

    # Create a new dictionary to hold the corrected weights
    from collections import OrderedDict
    corrected_weights = OrderedDict()
    
    # Iterate through the keys from the file and rename them
    for k, v in weights_from_file.items():
        # --- THIS IS THE FIX ---
        if k.startswith('module.'):
             # Remove 'module.' prefix if it exists
            new_key = k[7:]
        else:
            new_key = k
        
        # Rename the head layers
        if new_key == 'heads.classifier.weight':
            new_key = 'heads.weight'
        elif 'heads.bnneck.' in new_key:
            new_key = new_key.replace('heads.bnneck.', 'heads.bottleneck.0.')
        
        corrected_weights[new_key] = v
        # --------------------

    # Load the corrected weights into the model.
    # We use strict=False to ignore keys that are in the file but not needed by the model
    # (like pixel_mean, pixel_std).
    incompatible = model.load_state_dict(corrected_weights, strict=False)
    
    if not incompatible.missing_keys and not incompatible.unexpected_keys:
        print("✅ Success! All model weights were loaded correctly after manual correction.")
    else:
        print("❌ Error! Still some mismatches after correction.")
        print(f"Missing keys in model: {incompatible.missing_keys}")
        print(f"Unexpected keys in file: {incompatible.unexpected_keys}")

    print("--------------------------------\n")

    # --- Verification ---
    # ... (the rest of the script, like printing the model architecture) ...

    # --- Verification ---
    # Print the model architecture to verify it has been loaded correctly.
    # You should see the ResNet-50 blocks, the pooling layer, and the BNNeck.
    print("\n--- Model Architecture ---")
    print(model)
    print("--------------------------\n")
    
    print("FastReID model is ready for feature extraction.")
    
    # The 'model' object is now ready to be used for inference.
    return model

if __name__ == '__main__':
    loaded_model = main()

Configuration loaded successfully. Using device: cuda
Model built successfully.
Model set to evaluation mode.

--- Manually Correcting and Loading Weights ---
❌ Error! Still some mismatches after correction.
Missing keys in model: []
Unexpected keys in file: ['pixel_mean', 'pixel_std']
--------------------------------


--- Model Architecture ---
Baseline(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): IBN(
          (IN): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (BN): BatchNorm(32, eps=1e-05, momentum=0.1, affine=True, trac

In [7]:
import numpy

In [8]:
# reid_model='osnet_x0_25_msmt17.pt'

In [9]:
# results = model('img1.jpg', save=True)

In [4]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))  # Optional


True
NVIDIA GeForce RTX 4070 Laptop GPU


In [11]:
# vid = cv.VideoCapture("Double1.mp4")
# ptime=0
# while True:
#     isTrue, frame = vid.read()
#     if not isTrue:
#         print("End of video or failed to read frame.")
#         break
#     results = model.track(frame, verbose=False)  # device=0 for GPU

#     # Draw boxes manually
#     boxes = results[0].boxes.xyxy.cpu().numpy()  # [x1, y1, x2, y2]
#     confidences = results[0].boxes.conf.cpu().numpy()
#     class_ids = results[0].boxes.cls.cpu().numpy().astype(int)
#     names = model.names

#     for box, conf, cls in zip(boxes, confidences, class_ids):
#         x1, y1, x2, y2 = map(int, box)
#         label = f"{names[cls]} {conf:.2f}"
#         cv.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
#         cv.putText(frame, label, (x1, y1 - 10), cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

#     # FPS calculation
#     ctime = time.time()
#     fps = 1 / (ctime - ptime)
#     ptime = ctime
#     cv.putText(frame, f"FPS: {int(fps)}", (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
#     cv.imshow("frame", frame)
#     # ctime=time.time()
#     # fps=1/(ctime-ptime)
#     # ptime=ctime
#     # cv.putText(frame, str(int(fps)), (10, 70), cv.FONT_HERSHEY_PLAIN, 3, (0, 255, 0), 3)
#     # cv.imshow("frame",results[0].plot() )
#     # cv.imshow("frame", frame)
#     # Wait for 20ms, exit if 'd' is pressed
#     if cv.waitKey(1) & 0xFF == ord('d'):
#         break

# # Release the capture and destroy all windows
# vid.release()
# cv.destroyAllWindows()

In [12]:
# import cv2 as cv
# import time
# from ultralytics import YOLO

# # Load YOLOv8n model
# # model = YOLO("yolov8m.pt")  # you can also use yolov8m.pt etc.

# # Open video
# vid = cv.VideoCapture("palace.mp4")
# ptime = 0

# while True:
#     isTrue, frame = vid.read()
#     if not isTrue:
#         print("End of video or failed to read frame.")
#         break

#     # Inference (faster if you resize frame before)
#     results = model.track(frame, device=0, verbose=False,tracker='bytetrack.yaml',persist=True,classes=[0])  # device=0 for GPU

#     # Draw boxes manually
#     for r in results:
#         if r.boxes.id is not None:
#             boxes = r.boxes.xyxy.cpu().numpy()     # [x1, y1, x2, y2]
#             ids = r.boxes.id.cpu().numpy().astype(int)
#             cls = r.boxes.cls.cpu().numpy().astype(int)

#             for box, obj_id, class_id in zip(boxes, ids, cls):
#                 x1, y1, x2, y2 = map(int, box)
#                 # label = f"ID {obj_id} - {model.names[class_id]}"
#                 label = f"ID {obj_id} - {model.names[class_id]}"
#                 cv.rectangle(frame, (x1, y1), (x2, y2), (90, 189,87), 2)
#                 cv.putText(frame, label, (x1, y1 - 10), cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 1)
#     # FPS calculation
#     ctime = time.time()
#     fps = 1 / (ctime - ptime)
#     ptime = ctime
#     cv.putText(frame, f"FPS: {int(fps)}", (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

#     # Show frame
#     cv.imshow("YOLOv8 Detection", frame)

#     if cv.waitKey(100) & 0xFF == ord('d'):
#         break
   

# vid.release()
# cv.destroyAllWindows()


In [13]:
# ptime=0
# for result in model2.track(source=0, stream=True, tracker="bytetrack.yaml",persist=True,classes=[0]):
#     frame = result.orig_img

#     # Draw boxes with IDs
#     if result.boxes.id is not None:
#         boxes = result.boxes.xyxy.cpu().numpy()
#         ids = result.boxes.id.cpu().numpy().astype(int)
#         cls = result.boxes.cls.cpu().numpy().astype(int)
#         for box, obj_id, class_id in zip(boxes, ids, cls):
#             x1, y1, x2, y2 = map(int, box)
#             # label = f"ID {obj_id} - {model.names[class_id]}"
#             if len(boxes) >= 2:
#                 label = f"GAY No. {obj_id}"
#             else :
#                 label = f"Slave No. {obj_id}"
            
#             cv.rectangle(frame, (x1, y1), (x2, y2), (90, 189,87), 2)
#             cv.putText(frame, label, (x1, y1 - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
#     ctime = time.time()
#     fps = 1 / (ctime - ptime)
#     ptime = ctime
#     cv.putText(frame, f"FPS: {int(fps)}", (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
#     frame=cv.resize(frame,(1920,1080))
#     cv.imshow("Tracked", frame)
#     if cv.waitKey(1) == ord('q'):
        
#         break

# cv.destroyAllWindows()


In [14]:


# ptime=0

# for result in model3.track(source="palace.mp4", stream=True, tracker="bytetrack.yaml",persist=True,classes=[0],verbose=True):
#     frame = result.orig_img
#     # tracker_obj = model3.tracker
#     # print("Tracker class in use:", tracker_obj.__class__.__name__)
#     # with open("C:/Users/ayush/anaconda3/envs/fastreid_env/Lib/site-packages/ultralytics/cfg/trackers/botsort.yaml", "r") as f:
#     #     print(f.read())

#     # Draw boxes with IDs
#     if result.boxes.id is not None:
#         boxes = result.boxes.xyxy.cpu().numpy()
#         ids = result.boxes.id.cpu().numpy().astype(int)
#         cls = result.boxes.cls.cpu().numpy().astype(int)
#         for box, obj_id, class_id in zip(boxes, ids, cls):
#             x1, y1, x2, y2 = map(int, box)
#             # label = f"ID {obj_id} - {model2.names[class_id]}"
#             label = f"{obj_id}"
            
            
#             cv.rectangle(frame, (x1, y1), (x2, y2), (90, 189,87), 2)
#             cv.putText(frame, label, (x1, y1 - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
#     ctime = time.time()
#     fps = 1 / (ctime - ptime)
#     ptime = ctime
#     cv.putText(frame, f"FPS: {int(fps)}", (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    
#     cv.imshow("Tracked", frame)
#     if cv.waitKey(1) == ord('q'):
        
#         break

# cv.destroyAllWindows()


In [15]:
!pip install fastreid

  Using cached opencv_python_headless-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached torch-1.13.1-cp39-cp39-win_amd64.whl.metadata (23 kB)
  Using cached torchvision-0.14.1-cp39-cp39-win_amd64.whl.metadata (11 kB)
Using cached torch-1.13.1-cp39-cp39-win_amd64.whl (162.5 MB)
Using cached torchvision-0.14.1-cp39-cp39-win_amd64.whl (1.1 MB)
Using cached opencv_python_headless-4.12.0.88-cp37-abi3-win_amd64.whl (38.9 MB)

  Attempting uninstall: torch

    Found existing installation: torch 2.8.0+cu128

   ---------------------------------------- 0/3 [torch]
   ---------------------------------------- 0/3 [torch]
   ---------------------------------------- 0/3 [torch]
   ---------------------------------------- 0/3 [torch]
   ---------------------------------------- 0/3 [torch]
   ---------------------------------------- 0/3 [torch]
    Uninstalling torch-2.8.0+cu128:
   ---------------------------------------- 0/3 [torch]
      Successfully uninstalled torch-2.8.0+cu128


  You can safely remove it manually.
ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\ayush\\anaconda3\\envs\\tf\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



In [16]:
pip show fastreid

Name: fastreid
Version: 1.4.0
Summary: SOTA Re-identification Methods and Toolbox
Home-page: https://github.com/JDAI-CV/fast-reid
Author: He, Lingxiao and Liao, Xingyu and Liu, Wu and Liu, Xinchen and Cheng, Peng and Mei, Tao
Author-email: 
License: Apache 2.0
Location: c:\users\ayush\anaconda3\envs\tf\lib\site-packages
Requires: faiss-cpu, opencv-python-headless, scikit-learn, tabulate, tensorboard, termcolor, torch, torchvision, yacs
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [17]:

import fastreid
import torch

In [18]:
# from torch.nn.parallel import DistributedDataParallel, DataParallel
# import torch.nn as nn

# import copy
# import logging
# from collections.abc import MutableMapping
# from typing import Dict, Any
# class PathHandler:
#     """
#     PathHandler is a base class that defines common I/O functionality for a URI
#     protocol. It routes I/O for a generic URI which may look like "protocol://*"
#     or a canonical filepath "/foo/bar/baz".
#     """

#     _strict_kwargs_check = True
# class NativePathHandler(PathHandler):
#     """
#     Handles paths that can be accessed using Python native system calls. This
#     handler uses `open()` and `os.*` calls on the given path.
#     """
# class OrderedDict(dict):
#     'Dictionary that remembers insertion order'
#     # An inherited dict maps keys to values.
#     # The inherited dict provides __getitem__, __len__, __contains__, and get.
#     # The remaining methods are order-aware.
#     # Big-O running times for all methods are the same as regular dictionaries.

#     # The internal self.__map dict maps keys to links in a doubly linked list.
#     # The circular doubly linked list starts and ends with a sentinel element.
#     # The sentinel element never gets deleted (this simplifies the algorithm).
#     # The sentinel is in self.__hardroot with a weakref proxy in self.__root.
#     # The prev links are weakref proxies (to prevent circular references).
#     # Individual links are kept alive by the hard reference in self.__map.
#     # Those hard references disappear when a key is deleted from an OrderedDict.

# class PathHandler:
#     """
#     PathHandler is a base class that defines common I/O functionality for a URI
#     protocol. It routes I/O for a generic URI which may look like "protocol://*"
#     or a canonical filepath "/foo/bar/baz".
#     """

#     _strict_kwargs_check = True

#     def _check_kwargs(self, kwargs: Dict[str, Any]) -> None:
#         """
#         Checks if the given arguments are empty. Throws a ValueError if strict
#         kwargs checking is enabled and args are non-empty. If strict kwargs
#         checking is disabled, only a warning is logged.
#         Args:
#             kwargs (Dict[str, Any])
#         """
#         if self._strict_kwargs_check:
#             if len(kwargs) > 0:
#                 raise ValueError("Unused arguments: {}".format(kwargs))
#         else:
#             logger = logging.getLogger(__name__)
#             for k, v in kwargs.items():
#                 logger.warning(
#                     "[PathManager] {}={} argument ignored".format(k, v)
#                 )
# class PathManager:
#     """
#     A class for users to open generic paths or translate generic paths to file names.
#     """

#     _PATH_HANDLERS: MutableMapping[str, PathHandler] = OrderedDict()
#     _NATIVE_PATH_HANDLER = NativePathHandler()

#     @staticmethod
#     def __get_path_handler(path: str) -> PathHandler:
#         """
#         Finds a PathHandler that supports the given path. Falls back to the native
#         PathHandler if no other handler is found.
#         Args:
#             path (str): URI path to resource
#         Returns:
#             handler (PathHandler)
#         """
#         for p in PathManager._PATH_HANDLERS.keys():
#             if path.startswith(p):
#                 return PathManager._PATH_HANDLERS[p]
#         return PathManager._NATIVE_PATH_HANDLER

# class Checkpointer(object):
#     """
#     A checkpointer that can save/load model as well as extra checkpointable
#     objects.
#     """

#     def __init__(
#             self,
#             model: nn.Module,
#             save_dir: str = "",
#             *,
#             save_to_disk: bool = True,
#             **checkpointables: object,
#     ):
#         """
#         Args:
#             model (nn.Module): model.
#             save_dir (str): a directory to save and find checkpoints.
#             save_to_disk (bool): if True, save checkpoint to disk, otherwise
#                 disable saving for this checkpointer.
#             checkpointables (object): any checkpointable objects, i.e., objects
#                 that have the `state_dict()` and `load_state_dict()` method. For
#                 example, it can be used like
#                 `Checkpointer(model, "dir", optimizer=optimizer)`.
#         """
#         if isinstance(model, (DistributedDataParallel, DataParallel)):
#             model = model.module
#         self.model = model
#         self.checkpointables = copy.copy(checkpointables)
#         self.logger = logging.getLogger(__name__)
#         self.save_dir = save_dir
#         self.save_to_disk = save_to_disk

#         self.path_manager = PathManager

In [19]:
# def _type_check(arg, msg, is_argument=True, module=None, *, allow_special_forms=False):
#     """Check that the argument is a type, and return it (internal helper).

#     As a special case, accept None and return type(None) instead. Also wrap strings
#     into ForwardRef instances. Consider several corner cases, for example plain
#     special forms like Union are not valid, while Union[int, str] is OK, etc.
#     The msg argument is a human-readable error message, e.g.::

#         "Union[arg, ...]: arg should be a type."

#     We append the repr() of the actual value (truncated to 100 chars).
#     """
#     invalid_generic_forms = (Generic, Protocol)
#     if not allow_special_forms:
#         invalid_generic_forms += (ClassVar,)
#         if is_argument:
#             invalid_generic_forms += (Final,)

#     arg = _type_convert(arg, module=module, allow_special_forms=allow_special_forms)
#     if (isinstance(arg, _GenericAlias) and
#             arg.__origin__ in invalid_generic_forms):
#         raise TypeError(f"{arg} is not valid as type argument")
#     if arg in (Any, LiteralString, NoReturn, Never, Self, TypeAlias):
#         return arg
#     if allow_special_forms and arg in (ClassVar, Final):
#         return arg
#     if isinstance(arg, _SpecialForm) or arg in (Generic, Protocol):
#         raise TypeError(f"Plain {arg} is not valid as type argument")
#     if type(arg) is tuple:
#         raise TypeError(f"{msg} Got {arg!r:.100}.")
#     return arg
# def Optional(self, parameters):
#     """Optional[X] is equivalent to Union[X, None]."""
#     arg = _type_check(parameters, f"{self} requires a single type.")
#     return Union[arg, type(None)]

# def load(self, path: str, checkpointables: Optional[List[str]] = None) -> object:
#         """
#         Load from the given checkpoint. When path points to network file, this
#         function has to be called on all ranks.

#         Args:
#             path (str): path or url to the checkpoint. If empty, will not load
#                 anything.
#             checkpointables (list): List of checkpointable names to load. If not
#                 specified (None), will load all the possible checkpointables.
#         Returns:
#             dict:
#                 extra data loaded from the checkpoint that has not been
#                 processed. For example, those saved with
#                 :meth:`.save(**extra_data)`.
#         """
#         if not path:
#             # no checkpoint provided
#             self.logger.info("No checkpoint found. Training model from scratch")
#             return {}
#         self.logger.info("Loading checkpoint from {}".format(path))
#         if not os.path.isfile(path):
#             path = self.path_manager.get_local_path(path)
#             assert os.path.isfile(path), "Checkpoint {} not found!".format(path)

#         checkpoint = self._load_file(path)
#         incompatible = self._load_model(checkpoint)
#         if (
#                 incompatible is not None
#         ):  # handle some existing subclasses that returns None
#             self._log_incompatible_keys(incompatible)

#         for key in self.checkpointables if checkpointables is None else checkpointables:
#             if key in checkpoint:  # pyre-ignore
#                 self.logger.info("Loading {} from {}".format(key, path))
#                 obj = self.checkpointables[key]
#                 obj.load_state_dict(checkpoint.pop(key))  # pyre-ignore

#         # return any further checkpoint data
#         return checkpoint



In [20]:
# from fastreid.modeling import build_model


# class DefaultPredictor:
#     """
#     Create a simple end-to-end predictor with the given config.
#     The predictor takes an BGR image, resizes it to the specified resolution,
#     runs the model and produces a dict of predictions.
#     This predictor takes care of model loading and input preprocessing for you.
#     If you'd like to do anything more fancy, please refer to its source code
#     as examples to build and use the model manually.
#     Attributes:
#     Examples:
#     .. code-block:: python
#         pred = DefaultPredictor(cfg)
#         inputs = cv2.imread("input.jpg")
#         outputs = pred(inputs)
#     """

#     def __init__(self, cfg):
#         self.cfg = cfg.clone()  # cfg can be modified by model
#         self.cfg.defrost()
#         self.cfg.MODEL.BACKBONE.PRETRAIN = False
#         self.model = build_model(self.cfg)
#         self.model.eval()

#         Checkpointer(self.model).load(cfg.MODEL.WEIGHTS)

#     def __call__(self, image):
#         """
#         Args:
#             image (torch.tensor): an image tensor of shape (B, C, H, W).
#         Returns:
#             predictions (torch.tensor): the output features of the model
#         """
#         inputs = {"images": image.to(self.model.device)}
#         with torch.no_grad():  # https://github.com/sphinx-doc/sphinx/issues/4258
#             predictions = self.model(inputs)
#         return predictions.cpu()

In [21]:
# import sys
# sys.path.append("C:/CCTV/fast-reid/fastreid/engine")
# import defaults

In [22]:
#!/usr/bin/env python3
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved.

import copy
import logging
import os
from collections import defaultdict
from typing import Any
from typing import Optional, List, Dict, NamedTuple, Tuple, Iterable

import numpy as np
import torch
import torch.nn as nn
from termcolor import colored
from torch.nn.parallel import DistributedDataParallel, DataParallel

from fastreid.utils.file_io import PathManager


class _IncompatibleKeys(
    NamedTuple(
        # pyre-fixme[10]: Name `IncompatibleKeys` is used but not defined.
        "IncompatibleKeys",
        [
            ("missing_keys", List[str]),
            ("unexpected_keys", List[str]),
            # pyre-fixme[24]: Generic type `tuple` expects at least 1 type parameter.
            # pyre-fixme[24]: Generic type `tuple` expects at least 1 type parameter.
            # pyre-fixme[24]: Generic type `tuple` expects at least 1 type parameter.
            ("incorrect_shapes", List[Tuple]),
        ],
    )
):
    pass


class Checkpointer(object):
    """
    A checkpointer that can save/load model as well as extra checkpointable
    objects.
    """

    def __init__(
            self,
            model: nn.Module,
            save_dir: str = "",
            *,
            save_to_disk: bool = True,
            **checkpointables: object,
    ):
        """
        Args:
            model (nn.Module): model.
            save_dir (str): a directory to save and find checkpoints.
            save_to_disk (bool): if True, save checkpoint to disk, otherwise
                disable saving for this checkpointer.
            checkpointables (object): any checkpointable objects, i.e., objects
                that have the `state_dict()` and `load_state_dict()` method. For
                example, it can be used like
                `Checkpointer(model, "dir", optimizer=optimizer)`.
        """
        if isinstance(model, (DistributedDataParallel, DataParallel)):
            model = model.module
        self.model = model
        self.checkpointables = copy.copy(checkpointables)
        self.logger = logging.getLogger(__name__)
        self.save_dir = save_dir
        self.save_to_disk = save_to_disk

        self.path_manager = PathManager

    def save(self, name: str, **kwargs: Dict[str, str]):
        """
        Dump model and checkpointables to a file.

        Args:
            name (str): name of the file.
            kwargs (dict): extra arbitrary data to save.
        """
        if not self.save_dir or not self.save_to_disk:
            return

        data = {}
        data["model"] = self.model.state_dict()
        for key, obj in self.checkpointables.items():
            data[key] = obj.state_dict()
        data.update(kwargs)

        basename = "{}.pth".format(name)
        save_file = os.path.join(self.save_dir, basename)
        assert os.path.basename(save_file) == basename, basename
        self.logger.info("Saving checkpoint to {}".format(save_file))
        with PathManager.open(save_file, "wb") as f:
            torch.save(data, f)
        self.tag_last_checkpoint(basename)

    def load(self, path: str, checkpointables: Optional[List[str]] = None) -> object:
        """
        Load from the given checkpoint. When path points to network file, this
        function has to be called on all ranks.

        Args:
            path (str): path or url to the checkpoint. If empty, will not load
                anything.
            checkpointables (list): List of checkpointable names to load. If not
                specified (None), will load all the possible checkpointables.
        Returns:
            dict:
                extra data loaded from the checkpoint that has not been
                processed. For example, those saved with
                :meth:`.save(**extra_data)`.
        """
        if not path:
            # no checkpoint provided
            self.logger.info("No checkpoint found. Training model from scratch")
            return {}
        self.logger.info("Loading checkpoint from {}".format(path))
        if not os.path.isfile(path):
            path = self.path_manager.get_local_path(path)
            assert os.path.isfile(path), "Checkpoint {} not found!".format(path)

        checkpoint = self._load_file(path)
        incompatible = self._load_model(checkpoint)
        if (
                incompatible is not None
        ):  # handle some existing subclasses that returns None
            self._log_incompatible_keys(incompatible)

        for key in self.checkpointables if checkpointables is None else checkpointables:
            if key in checkpoint:  # pyre-ignore
                self.logger.info("Loading {} from {}".format(key, path))
                obj = self.checkpointables[key]
                obj.load_state_dict(checkpoint.pop(key))  # pyre-ignore

        # return any further checkpoint data
        return checkpoint

    def has_checkpoint(self):
        """
        Returns:
            bool: whether a checkpoint exists in the target directory.
        """
        save_file = os.path.join(self.save_dir, "last_checkpoint")
        return PathManager.exists(save_file)

    def get_checkpoint_file(self):
        """
        Returns:
            str: The latest checkpoint file in target directory.
        """
        save_file = os.path.join(self.save_dir, "last_checkpoint")
        try:
            with PathManager.open(save_file, "r") as f:
                last_saved = f.read().strip()
        except IOError:
            # if file doesn't exist, maybe because it has just been
            # deleted by a separate process
            return ""
        return os.path.join(self.save_dir, last_saved)

    def get_all_checkpoint_files(self):
        """
        Returns:
            list: All available checkpoint files (.pth files) in target
                directory.
        """
        all_model_checkpoints = [
            os.path.join(self.save_dir, file)
            for file in PathManager.ls(self.save_dir)
            if PathManager.isfile(os.path.join(self.save_dir, file))
               and file.endswith(".pth")
        ]
        return all_model_checkpoints

    def resume_or_load(self, path: str, *, resume: bool = True):
        """
        If `resume` is True, this method attempts to resume from the last
        checkpoint, if exists. Otherwise, load checkpoint from the given path.
        This is useful when restarting an interrupted training job.

        Args:
            path (str): path to the checkpoint.
            resume (bool): if True, resume from the last checkpoint if it exists.
        Returns:
            same as :meth:`load`.
        """
        if resume and self.has_checkpoint():
            path = self.get_checkpoint_file()
            return self.load(path)
        else:
            return self.load(path, checkpointables=[])

    def tag_last_checkpoint(self, last_filename_basename: str):
        """
        Tag the last checkpoint.

        Args:
            last_filename_basename (str): the basename of the last filename.
        """
        save_file = os.path.join(self.save_dir, "last_checkpoint")
        with PathManager.open(save_file, "w") as f:
            f.write(last_filename_basename)

    def _load_file(self, f: str):
        """
        Load a checkpoint file. Can be overwritten by subclasses to support
        different formats.

        Args:
            f (str): a locally mounted file path.
        Returns:
            dict: with keys "model" and optionally others that are saved by
                the checkpointer dict["model"] must be a dict which maps strings
                to torch.Tensor or numpy arrays.
        """
        return torch.load(f, map_location=torch.device("cpu"))

    def _load_model(self, checkpoint: Any):
        """
        Load weights from a checkpoint.

        Args:
            checkpoint (Any): checkpoint contains the weights.
        """
        checkpoint_state_dict = checkpoint.pop("model")
        self._convert_ndarray_to_tensor(checkpoint_state_dict)

        # if the state_dict comes from a model that was wrapped in a
        # DataParallel or DistributedDataParallel during serialization,
        # remove the "module" prefix before performing the matching.
        _strip_prefix_if_present(checkpoint_state_dict, "module.")

        # work around https://github.com/pytorch/pytorch/issues/24139
        model_state_dict = self.model.state_dict()
        incorrect_shapes = []
        for k in list(checkpoint_state_dict.keys()):
            if k in model_state_dict:
                shape_model = tuple(model_state_dict[k].shape)
                shape_checkpoint = tuple(checkpoint_state_dict[k].shape)
                if shape_model != shape_checkpoint:
                    incorrect_shapes.append((k, shape_checkpoint, shape_model))
                    checkpoint_state_dict.pop(k)

        incompatible = self.model.load_state_dict(checkpoint_state_dict, strict=False)
        return _IncompatibleKeys(
            missing_keys=incompatible.missing_keys,
            unexpected_keys=incompatible.unexpected_keys,
            incorrect_shapes=incorrect_shapes,
        )

    def _log_incompatible_keys(self, incompatible: _IncompatibleKeys) -> None:
        """
        Log information about the incompatible keys returned by ``_load_model``.
        """
        for k, shape_checkpoint, shape_model in incompatible.incorrect_shapes:
            self.logger.warning(
                "Skip loading parameter '{}' to the model due to incompatible "
                "shapes: {} in the checkpoint but {} in the "
                "model! You might want to double check if this is expected.".format(
                    k, shape_checkpoint, shape_model
                )
            )
        if incompatible.missing_keys:
            missing_keys = _filter_reused_missing_keys(
                self.model, incompatible.missing_keys
            )
            if missing_keys:
                self.logger.info(get_missing_parameters_message(missing_keys))
        if incompatible.unexpected_keys:
            self.logger.info(
                get_unexpected_parameters_message(incompatible.unexpected_keys)
            )

    def _convert_ndarray_to_tensor(self, state_dict: dict):
        """
        In-place convert all numpy arrays in the state_dict to torch tensor.

        Args:
            state_dict (dict): a state-dict to be loaded to the model.
        """
        # model could be an OrderedDict with _metadata attribute
        # (as returned by Pytorch's state_dict()). We should preserve these
        # properties.
        for k in list(state_dict.keys()):
            v = state_dict[k]
            if not isinstance(v, np.ndarray) and not isinstance(
                    v, torch.Tensor
            ):
                raise ValueError(
                    "Unsupported type found in checkpoint! {}: {}".format(
                        k, type(v)
                    )
                )
            if not isinstance(v, torch.Tensor):
                state_dict[k] = torch.from_numpy(v)


class PeriodicCheckpointer:
    """
    Save checkpoints periodically. When `.step(iteration)` is called, it will
    execute `checkpointer.save` on the given checkpointer, if iteration is a
    multiple of period or if `max_iter` is reached.
    """

    def __init__(self, checkpointer: Any, period: int, max_epoch: int = None):
        """
        Args:
            checkpointer (Any): the checkpointer object used to save
            checkpoints.
            period (int): the period to save checkpoint.
            max_epoch (int): maximum number of epochs. When it is reached,
                a checkpoint named "model_final" will be saved.
        """
        self.checkpointer = checkpointer
        self.period = int(period)
        self.max_epoch = max_epoch
        self.best_metric = -1

    def step(self, epoch: int, **kwargs: Any):
        """
        Perform the appropriate action at the given iteration.

        Args:
            epoch (int): the current epoch, ranged in [0, max_epoch-1].
            kwargs (Any): extra data to save, same as in
                :meth:`Checkpointer.save`.
        """
        epoch = int(epoch)
        additional_state = {"epoch": epoch}
        additional_state.update(kwargs)
        if (epoch + 1) % self.period == 0 and epoch < self.max_epoch - 1:
            if additional_state["metric"] > self.best_metric:
                self.checkpointer.save(
                    "model_best", **additional_state
                )
                self.best_metric = additional_state["metric"]
            # Put it behind best model save to make last checkpoint valid
            self.checkpointer.save(
                "model_{:04d}".format(epoch), **additional_state
            )
        if epoch >= self.max_epoch - 1:
            if additional_state["metric"] > self.best_metric:
                self.checkpointer.save(
                    "model_best", **additional_state
                )
            self.checkpointer.save("model_final", **additional_state)

    def save(self, name: str, **kwargs: Any):
        """
        Same argument as :meth:`Checkpointer.save`.
        Use this method to manually save checkpoints outside the schedule.

        Args:
            name (str): file name.
            kwargs (Any): extra data to save, same as in
                :meth:`Checkpointer.save`.
        """
        self.checkpointer.save(name, **kwargs)


def _filter_reused_missing_keys(model: nn.Module, keys: List[str]) -> List[str]:
    """
    Filter "missing keys" to not include keys that have been loaded with another name.
    """
    keyset = set(keys)
    param_to_names = defaultdict(set)  # param -> names that points to it
    for module_prefix, module in _named_modules_with_dup(model):
        for name, param in list(module.named_parameters(recurse=False)) + list(
                module.named_buffers(recurse=False)  # pyre-ignore
        ):
            full_name = (module_prefix + "." if module_prefix else "") + name
            param_to_names[param].add(full_name)
    for names in param_to_names.values():
        # if one name appears missing but its alias exists, then this
        # name is not considered missing
        if any(n in keyset for n in names) and not all(n in keyset for n in names):
            [keyset.remove(n) for n in names if n in keyset]
    return list(keyset)


def get_missing_parameters_message(keys: List[str]) -> str:
    """
    Get a logging-friendly message to report parameter names (keys) that are in
    the model but not found in a checkpoint.

    Args:
        keys (list[str]): List of keys that were not found in the checkpoint.
    Returns:
        str: message.
    """
    groups = _group_checkpoint_keys(keys)
    msg = "Some model parameters or buffers are not found in the checkpoint:\n"
    msg += "\n".join(
        "  " + colored(k + _group_to_str(v), "blue") for k, v in groups.items()
    )
    return msg


def get_unexpected_parameters_message(keys: List[str]) -> str:
    """
    Get a logging-friendly message to report parameter names (keys) that are in
    the checkpoint but not found in the model.

    Args:
        keys (list[str]): List of keys that were not found in the model.
    Returns:
        str: message.
    """
    groups = _group_checkpoint_keys(keys)
    msg = "The checkpoint state_dict contains keys that are not used by the model:\n"
    msg += "\n".join(
        "  " + colored(k + _group_to_str(v), "magenta") for k, v in groups.items()
    )
    return msg


def _strip_prefix_if_present(state_dict: Dict[str, Any], prefix: str) -> None:
    """
    Strip the prefix in metadata, if any.

    Args:
        state_dict (OrderedDict): a state-dict to be loaded to the model.
        prefix (str): prefix.
    """
    keys = sorted(state_dict.keys())
    if not all(len(key) == 0 or key.startswith(prefix) for key in keys):
        return

    for key in keys:
        newkey = key[len(prefix):]
        state_dict[newkey] = state_dict.pop(key)

    # also strip the prefix in metadata, if any..
    try:
        metadata = state_dict._metadata  # pyre-ignore
    except AttributeError:
        pass
    else:
        for key in list(metadata.keys()):
            # for the metadata dict, the key can be:
            # '': for the DDP module, which we want to remove.
            # 'module': for the actual model.
            # 'module.xx.xx': for the rest.

            if len(key) == 0:
                continue
            newkey = key[len(prefix):]
            metadata[newkey] = metadata.pop(key)


def _group_checkpoint_keys(keys: List[str]) -> Dict[str, List[str]]:
    """
    Group keys based on common prefixes. A prefix is the string up to the final
    "." in each key.

    Args:
        keys (list[str]): list of parameter names, i.e. keys in the model
            checkpoint dict.
    Returns:
        dict[list]: keys with common prefixes are grouped into lists.
    """
    groups = defaultdict(list)
    for key in keys:
        pos = key.rfind(".")
        if pos >= 0:
            head, tail = key[:pos], [key[pos + 1:]]
        else:
            head, tail = key, []
        groups[head].extend(tail)
    return groups


def _group_to_str(group: List[str]) -> str:
    """
    Format a group of parameter name suffixes into a loggable string.

    Args:
        group (list[str]): list of parameter name suffixes.
    Returns:
        str: formated string.
    """
    if len(group) == 0:
        return ""

    if len(group) == 1:
        return "." + group[0]

    return ".{" + ", ".join(group) + "}"


def _named_modules_with_dup(
        model: nn.Module, prefix: str = ""
) -> Iterable[Tuple[str, nn.Module]]:
    """
    The same as `model.named_modules()`, except that it includes
    duplicated modules that have more than one name.
    """
    yield prefix, model
    for name, module in model._modules.items():  # pyre-ignore
        if module is None:
            continue
        submodule_prefix = prefix + ("." if prefix else "") + name
        yield from _named_modules_with_dup(module, submodule_prefix)


In [23]:
# encoding: utf-8
#train-loop
"""
credit:
https://github.com/facebookresearch/detectron2/blob/master/detectron2/engine/train_loop.py
"""

import logging
import time
import weakref
from typing import Dict

import numpy as np
import torch
from torch.nn.parallel import DataParallel, DistributedDataParallel

import fastreid.utils.comm as comm
from fastreid.utils.events import EventStorage, get_event_storage
from fastreid.utils.params import ContiguousParams

__all__ = ["HookBase", "TrainerBase", "SimpleTrainer"]

logger = logging.getLogger(__name__)


class HookBase:
    """
    Base class for hooks that can be registered with :class:`TrainerBase`.
    Each hook can implement 6 methods. The way they are called is demonstrated
    in the following snippet:
    .. code-block:: python
        hook.before_train()
        for _ in range(start_epoch, max_epoch):
            hook.before_epoch()
            for iter in range(start_iter, max_iter):
                hook.before_step()
                trainer.run_step()
                hook.after_step()
            hook.after_epoch()
        hook.after_train()
    Notes:
        1. In the hook method, users can access `self.trainer` to access more
           properties about the context (e.g., current iteration).
        2. A hook that does something in :meth:`before_step` can often be
           implemented equivalently in :meth:`after_step`.
           If the hook takes non-trivial time, it is strongly recommended to
           implement the hook in :meth:`after_step` instead of :meth:`before_step`.
           The convention is that :meth:`before_step` should only take negligible time.
           Following this convention will allow hooks that do care about the difference
           between :meth:`before_step` and :meth:`after_step` (e.g., timer) to
           function properly.
    Attributes:
        trainer: A weak reference to the trainer object. Set by the trainer when the hook is
            registered.
    """

    def before_train(self):
        """
        Called before the first iteration.
        """
        pass

    def after_train(self):
        """
        Called after the last iteration.
        """
        pass

    def before_epoch(self):
        """
        Called before each epoch.
        """
        pass

    def after_epoch(self):
        """
        Called after each epoch.
        """
        pass

    def before_step(self):
        """
        Called before each iteration.
        """
        pass

    def after_step(self):
        """
        Called after each iteration.
        """
        pass


class TrainerBase:
    """
    Base class for iterative trainer with hooks.
    The only assumption we made here is: the training runs in a loop.
    A subclass can implement what the loop is.
    We made no assumptions about the existence of dataloader, optimizer, model, etc.
    Attributes:
        iter(int): the current iteration.
        epoch(int): the current epoch.
        start_iter(int): The iteration to start with.
            By convention the minimum possible value is 0.
        max_epoch (int): The epoch to end training.
        storage(EventStorage): An EventStorage that's opened during the course of training.
    """

    def __init__(self):
        self._hooks = []

    def register_hooks(self, hooks):
        """
        Register hooks to the trainer. The hooks are executed in the order
        they are registered.
        Args:
            hooks (list[Optional[HookBase]]): list of hooks
        """
        hooks = [h for h in hooks if h is not None]
        for h in hooks:
            assert isinstance(h, HookBase)
            # To avoid circular reference, hooks and trainer cannot own each other.
            # This normally does not matter, but will cause memory leak if the
            # involved objects contain __del__:
            # See http://engineering.hearsaysocial.com/2013/06/16/circular-references-in-python/
            h.trainer = weakref.proxy(self)
        self._hooks.extend(hooks)

    def train(self, start_epoch: int, max_epoch: int, iters_per_epoch: int):
        """
        Args:
            start_epoch, max_epoch (int): See docs above
        """
        logger = logging.getLogger(__name__)
        logger.info("Starting training from epoch {}".format(start_epoch))

        self.iter = self.start_iter = start_epoch * iters_per_epoch

        with EventStorage(self.start_iter) as self.storage:
            try:
                self.before_train()
                for self.epoch in range(start_epoch, max_epoch):
                    self.before_epoch()
                    for _ in range(iters_per_epoch):
                        self.before_step()
                        self.run_step()
                        self.after_step()
                        self.iter += 1
                    self.after_epoch()
            except Exception:
                logger.exception("Exception during training:")
                raise
            finally:
                self.after_train()

    def before_train(self):
        for h in self._hooks:
            h.before_train()

    def after_train(self):
        self.storage.iter = self.iter
        for h in self._hooks:
            h.after_train()

    def before_epoch(self):
        self.storage.epoch = self.epoch

        for h in self._hooks:
            h.before_epoch()

    def before_step(self):
        self.storage.iter = self.iter

        for h in self._hooks:
            h.before_step()

    def after_step(self):
        for h in self._hooks:
            h.after_step()

    def after_epoch(self):
        for h in self._hooks:
            h.after_epoch()

    def run_step(self):
        raise NotImplementedError


class SimpleTrainer(TrainerBase):
    """
    A simple trainer for the most common type of task:
    single-cost single-optimizer single-data-source iterative optimization.
    It assumes that every step, you:
    1. Compute the loss with a data from the data_loader.
    2. Compute the gradients with the above loss.
    3. Update the model with the optimizer.
    If you want to do anything fancier than this,
    either subclass TrainerBase and implement your own `run_step`,
    or write your own training loop.
    """

    def __init__(self, model, data_loader, optimizer, param_wrapper):
        """
        Args:
            model: a torch Module. Takes a data from data_loader and returns a
                dict of heads.
            data_loader: an iterable. Contains data to be used to call model.
            optimizer: a torch optimizer.
        """
        super().__init__()

        """
        We set the model to training mode in the trainer.
        However it's valid to train a model that's in eval mode.
        If you want your model (or a submodule of it) to behave
        like evaluation during training, you can overwrite its train() method.
        """
        model.train()

        self.model = model
        self.data_loader = data_loader
        self._data_loader_iter = iter(data_loader)
        self.optimizer = optimizer
        self.param_wrapper = param_wrapper

    def run_step(self):
        """
        Implement the standard training logic described above.
        """
        assert self.model.training, "[SimpleTrainer] model was changed to eval mode!"
        start = time.perf_counter()
        """
        If your want to do something with the data, you can wrap the dataloader.
        """
        data = next(self._data_loader_iter)
        data_time = time.perf_counter() - start

        """
        If your want to do something with the heads, you can wrap the model.
        """

        loss_dict = self.model(data)
        losses = sum(loss_dict.values())

        """
        If you need accumulate gradients or something similar, you can
        wrap the optimizer with your custom `zero_grad()` method.
        """
        self.optimizer.zero_grad()

        losses.backward()

        self._write_metrics(loss_dict, data_time)

        """
        If you need gradient clipping/scaling or other processing, you can
        wrap the optimizer with your custom `step()` method.
        """
        self.optimizer.step()
        if isinstance(self.param_wrapper, ContiguousParams):
            self.param_wrapper.assert_buffer_is_valid()

    def _write_metrics(self, loss_dict: Dict[str, torch.Tensor], data_time: float):
        """
        Args:
            loss_dict (dict): dict of scalar losses
            data_time (float): time taken by the dataloader iteration
        """
        device = next(iter(loss_dict.values())).device

        # Use a new stream so these ops don't wait for DDP or backward
        with torch.cuda.stream(torch.cuda.Stream() if device.type == "cuda" else None):
            metrics_dict = {k: v.detach().cpu().item() for k, v in loss_dict.items()}
            metrics_dict["data_time"] = data_time

            # Gather metrics among all workers for logging
            # This assumes we do DDP-style training, which is currently the only
            # supported method in detectron2.
            all_metrics_dict = comm.gather(metrics_dict)

        if comm.is_main_process():
            storage = get_event_storage()

            # data_time among workers can have high variance. The actual latency
            # caused by data_time is the maximum among workers.
            data_time = np.max([x.pop("data_time") for x in all_metrics_dict])
            storage.put_scalar("data_time", data_time)

            # average the rest metrics
            metrics_dict = {
                k: np.mean([x[k] for x in all_metrics_dict]) for k in all_metrics_dict[0].keys()
            }
            total_losses_reduced = sum(metrics_dict.values())
            if not np.isfinite(total_losses_reduced):
                raise FloatingPointError(
                    f"Loss became infinite or NaN at iteration={self.iter}!\n"
                    f"loss_dict = {metrics_dict}"
                )

            storage.put_scalar("total_loss", total_losses_reduced)
            if len(metrics_dict) > 1:
                storage.put_scalars(**metrics_dict)


class AMPTrainer(SimpleTrainer):
    """
    Like :class:`SimpleTrainer`, but uses automatic mixed precision
    in the training loop.
    """

    def __init__(self, model, data_loader, optimizer, param_wrapper, grad_scaler=None):
        """

        Args:
            model, data_loader, optimizer: same as in :class:`SimpleTrainer`.
            grad_scaler: torch GradScaler to automatically scale gradients.
        """
        unsupported = "AMPTrainer does not support single-process multi-device training!"
        if isinstance(model, DistributedDataParallel):
            assert not (model.device_ids and len(model.device_ids) > 1), unsupported
        assert not isinstance(model, DataParallel), unsupported

        super().__init__(model, data_loader, optimizer, param_wrapper)

        if grad_scaler is None:
            from torch.cuda.amp import GradScaler

            grad_scaler = GradScaler()
        self.grad_scaler = grad_scaler

    def run_step(self):
        """
        Implement the AMP training logic.
        """
        assert self.model.training, "[AMPTrainer] model was changed to eval mode!"
        assert torch.cuda.is_available(), "[AMPTrainer] CUDA is required for AMP training!"
        from torch.cuda.amp import autocast

        start = time.perf_counter()
        data = next(self._data_loader_iter)
        data_time = time.perf_counter() - start

        with autocast():
            loss_dict = self.model(data)
            losses = sum(loss_dict.values())

        self.optimizer.zero_grad()
        self.grad_scaler.scale(losses).backward()

        self._write_metrics(loss_dict, data_time)

        self.grad_scaler.step(self.optimizer)
        self.grad_scaler.update()
        if isinstance(self.param_wrapper, ContiguousParams):
            self.param_wrapper.assert_buffer_is_valid()


In [24]:
# -*- coding: utf-8 -*-
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved

import datetime
import itertools
import logging
import os
import tempfile
import time
from collections import Counter

import torch
from torch import nn
from torch.nn.parallel import DistributedDataParallel

from fastreid.evaluation.testing import flatten_results_dict
import sys
sys.path.append("C:\CCTV\fast-reid\fastreid\solver\optim")
# from fastreid.solver import optim
from fastreid.utils import comm
from fastreid.utils.checkpoint import PeriodicCheckpointer as _PeriodicCheckpointer
from fastreid.utils.events import EventStorage, EventWriter, get_event_storage
from fastreid.utils.file_io import PathManager
from fastreid.utils.precision_bn import update_bn_stats, get_bn_modules
from fastreid.utils.timer import Timer
# from .train_loop import HookBase

__all__ = [
    "CallbackHook",
    "IterationTimer",
    "PeriodicWriter",
    "PeriodicCheckpointer",
    "LRScheduler",
    "AutogradProfiler",
    "EvalHook",
    "PreciseBN",
    "LayerFreeze",
]

"""
Implement some common hooks.
"""


class CallbackHook(HookBase):
    """
    Create a hook using callback functions provided by the user.
    """

    def __init__(self, *, before_train=None, after_train=None, before_epoch=None, after_epoch=None,
                 before_step=None, after_step=None):
        """
        Each argument is a function that takes one argument: the trainer.
        """
        self._before_train = before_train
        self._before_epoch = before_epoch
        self._before_step = before_step
        self._after_step = after_step
        self._after_epoch = after_epoch
        self._after_train = after_train

    def before_train(self):
        if self._before_train:
            self._before_train(self.trainer)

    def after_train(self):
        if self._after_train:
            self._after_train(self.trainer)
        # The functions may be closures that hold reference to the trainer
        # Therefore, delete them to avoid circular reference.
        del self._before_train, self._after_train
        del self._before_step, self._after_step

    def before_epoch(self):
        if self._before_epoch:
            self._before_epoch(self.trainer)

    def after_epoch(self):
        if self._after_epoch:
            self._after_epoch(self.trainer)

    def before_step(self):
        if self._before_step:
            self._before_step(self.trainer)

    def after_step(self):
        if self._after_step:
            self._after_step(self.trainer)


class IterationTimer(HookBase):
    """
    Track the time spent for each iteration (each run_step call in the trainer).
    Print a summary in the end of training.
    This hook uses the time between the call to its :meth:`before_step`
    and :meth:`after_step` methods.
    Under the convention that :meth:`before_step` of all hooks should only
    take negligible amount of time, the :class:`IterationTimer` hook should be
    placed at the beginning of the list of hooks to obtain accurate timing.
    """

    def __init__(self, warmup_iter=3):
        """
        Args:
            warmup_iter (int): the number of iterations at the beginning to exclude
                from timing.
        """
        self._warmup_iter = warmup_iter
        self._step_timer = Timer()

    def before_train(self):
        self._start_time = time.perf_counter()
        self._total_timer = Timer()
        self._total_timer.pause()

    def after_train(self):
        logger = logging.getLogger(__name__)
        total_time = time.perf_counter() - self._start_time
        total_time_minus_hooks = self._total_timer.seconds()
        hook_time = total_time - total_time_minus_hooks

        num_iter = self.trainer.iter + 1 - self.trainer.start_iter - self._warmup_iter

        if num_iter > 0 and total_time_minus_hooks > 0:
            # Speed is meaningful only after warmup
            # NOTE this format is parsed by grep in some scripts
            logger.info(
                "Overall training speed: {} iterations in {} ({:.4f} s / it)".format(
                    num_iter,
                    str(datetime.timedelta(seconds=int(total_time_minus_hooks))),
                    total_time_minus_hooks / num_iter,
                )
            )

        logger.info(
            "Total training time: {} ({} on hooks)".format(
                str(datetime.timedelta(seconds=int(total_time))),
                str(datetime.timedelta(seconds=int(hook_time))),
            )
        )

    def before_step(self):
        self._step_timer.reset()
        self._total_timer.resume()

    def after_step(self):
        # +1 because we're in after_step
        iter_done = self.trainer.iter - self.trainer.start_iter + 1
        if iter_done >= self._warmup_iter:
            sec = self._step_timer.seconds()
            self.trainer.storage.put_scalars(time=sec)
        else:
            self._start_time = time.perf_counter()
            self._total_timer.reset()

        self._total_timer.pause()


class PeriodicWriter(HookBase):
    """
    Write events to EventStorage periodically.
    It is executed every ``period`` iterations and after the last iteration.
    """

    def __init__(self, writers, period=20):
        """
        Args:
            writers (list[EventWriter]): a list of EventWriter objects
            period (int):
        """
        self._writers = writers
        for w in writers:
            assert isinstance(w, EventWriter), w
        self._period = period

    def after_step(self):
        if (self.trainer.iter + 1) % self._period == 0 or (
                self.trainer.iter == self.trainer.max_iter - 1
        ):
            for writer in self._writers:
                writer.write()

    def after_epoch(self):
        for writer in self._writers:
            writer.write()

    def after_train(self):
        for writer in self._writers:
            writer.close()


class PeriodicCheckpointer(_PeriodicCheckpointer, HookBase):
    """
    Same as :class:`fastreid.utils.checkpoint.PeriodicCheckpointer`, but as a hook.
    Note that when used as a hook,
    it is unable to save additional data other than what's defined
    by the given `checkpointer`.
    It is executed every ``period`` iterations and after the last iteration.
    """

    def before_train(self):
        self.max_epoch = self.trainer.max_epoch
        if len(self.trainer.cfg.DATASETS.TESTS) == 1:
            self.metric_name = "metric"
        else:
            self.metric_name = self.trainer.cfg.DATASETS.TESTS[0] + "/metric"

    def after_epoch(self):
        # No way to use **kwargs
        storage = get_event_storage()
        metric_dict = dict(
            metric=storage.latest()[self.metric_name][0] if self.metric_name in storage.latest() else -1
        )
        self.step(self.trainer.epoch, **metric_dict)


class LRScheduler(HookBase):
    """
    A hook which executes a torch builtin LR scheduler and summarizes the LR.
    It is executed after every iteration.
    """

    def __init__(self, optimizer, scheduler):
        """
        Args:
            optimizer (torch.optim.Optimizer):
            scheduler (torch.optim._LRScheduler)
        """
        self._optimizer = optimizer
        self._scheduler = scheduler
        self._scale = 0

        # NOTE: some heuristics on what LR to summarize
        # summarize the param group with most parameters
        largest_group = max(len(g["params"]) for g in optimizer.param_groups)

        if largest_group == 1:
            # If all groups have one parameter,
            # then find the most common initial LR, and use it for summary
            lr_count = Counter([g["lr"] for g in optimizer.param_groups])
            lr = lr_count.most_common()[0][0]
            for i, g in enumerate(optimizer.param_groups):
                if g["lr"] == lr:
                    self._best_param_group_id = i
                    break
        else:
            for i, g in enumerate(optimizer.param_groups):
                if len(g["params"]) == largest_group:
                    self._best_param_group_id = i
                    break

    def before_step(self):
        if self.trainer.grad_scaler is not None:
            self._scale = self.trainer.grad_scaler.get_scale()

    def after_step(self):
        lr = self._optimizer.param_groups[self._best_param_group_id]["lr"]
        self.trainer.storage.put_scalar("lr", lr, smoothing_hint=False)

        next_iter = self.trainer.iter + 1
        if next_iter <= self.trainer.warmup_iters:
            if self.trainer.grad_scaler is None or self._scale == self.trainer.grad_scaler.get_scale():
                self._scheduler["warmup_sched"].step()

    def after_epoch(self):
        next_iter = self.trainer.iter + 1
        next_epoch = self.trainer.epoch + 1
        if next_iter > self.trainer.warmup_iters and next_epoch > self.trainer.delay_epochs:
            self._scheduler["lr_sched"].step()


class AutogradProfiler(HookBase):
    """
    A hook which runs `torch.autograd.profiler.profile`.
    Examples:
    .. code-block:: python
        hooks.AutogradProfiler(
             lambda trainer: trainer.iter > 10 and trainer.iter < 20, self.cfg.OUTPUT_DIR
        )
    The above example will run the profiler for iteration 10~20 and dump
    results to ``OUTPUT_DIR``. We did not profile the first few iterations
    because they are typically slower than the rest.
    The result files can be loaded in the ``chrome://tracing`` page in chrome browser.
    Note:
        When used together with NCCL on older version of GPUs,
        autograd profiler may cause deadlock because it unnecessarily allocates
        memory on every device it sees. The memory management calls, if
        interleaved with NCCL calls, lead to deadlock on GPUs that do not
        support `cudaLaunchCooperativeKernelMultiDevice`.
    """

    def __init__(self, enable_predicate, output_dir, *, use_cuda=True):
        """
        Args:
            enable_predicate (callable[trainer -> bool]): a function which takes a trainer,
                and returns whether to enable the profiler.
                It will be called once every step, and can be used to select which steps to profile.
            output_dir (str): the output directory to dump tracing files.
            use_cuda (bool): same as in `torch.autograd.profiler.profile`.
        """
        self._enable_predicate = enable_predicate
        self._use_cuda = use_cuda
        self._output_dir = output_dir

    def before_step(self):
        if self._enable_predicate(self.trainer):
            self._profiler = torch.autograd.profiler.profile(use_cuda=self._use_cuda)
            self._profiler.__enter__()
        else:
            self._profiler = None

    def after_step(self):
        if self._profiler is None:
            return
        self._profiler.__exit__(None, None, None)
        out_file = os.path.join(
            self._output_dir, "profiler-trace-iter{}.json".format(self.trainer.iter)
        )
        if "://" not in out_file:
            self._profiler.export_chrome_trace(out_file)
        else:
            # Support non-posix filesystems
            with tempfile.TemporaryDirectory(prefix="fastreid_profiler") as d:
                tmp_file = os.path.join(d, "tmp.json")
                self._profiler.export_chrome_trace(tmp_file)
                with open(tmp_file) as f:
                    content = f.read()
            with PathManager.open(out_file, "w") as f:
                f.write(content)


class EvalHook(HookBase):
    """
    Run an evaluation function periodically, and at the end of training.
    It is executed every ``eval_period`` iterations and after the last iteration.
    """

    def __init__(self, eval_period, eval_function):
        """
        Args:
            eval_period (int): the period to run `eval_function`.
            eval_function (callable): a function which takes no arguments, and
                returns a nested dict of evaluation metrics.
        Note:
            This hook must be enabled in all or none workers.
            If you would like only certain workers to perform evaluation,
            give other workers a no-op function (`eval_function=lambda: None`).
        """
        self._period = eval_period
        self._func = eval_function

    def _do_eval(self):
        results = self._func()

        if results:
            assert isinstance(
                results, dict
            ), "Eval function must return a dict. Got {} instead.".format(results)

            flattened_results = flatten_results_dict(results)
            for k, v in flattened_results.items():
                try:
                    v = float(v)
                except Exception:
                    raise ValueError(
                        "[EvalHook] eval_function should return a nested dict of float. "
                        "Got '{}: {}' instead.".format(k, v)
                    )
            self.trainer.storage.put_scalars(**flattened_results, smoothing_hint=False)

        torch.cuda.empty_cache()
        # Evaluation may take different time among workers.
        # A barrier make them start the next iteration together.
        comm.synchronize()

    def after_epoch(self):
        next_epoch = self.trainer.epoch + 1
        if self._period > 0 and next_epoch % self._period == 0:
            self._do_eval()

    def after_train(self):
        next_epoch = self.trainer.epoch + 1
        # This condition is to prevent the eval from running after a failed training
        if next_epoch % self._period != 0 and next_epoch >= self.trainer.max_epoch:
            self._do_eval()
        # func is likely a closure that holds reference to the trainer
        # therefore we clean it to avoid circular reference in the end
        del self._func


class PreciseBN(HookBase):
    """
    The standard implementation of BatchNorm uses EMA in inference, which is
    sometimes suboptimal.
    This class computes the true average of statistics rather than the moving average,
    and put true averages to every BN layer in the given model.
    It is executed after the last iteration.
    """

    def __init__(self, model, data_loader, num_iter):
        """
        Args:
            model (nn.Module): a module whose all BN layers in training mode will be
                updated by precise BN.
                Note that user is responsible for ensuring the BN layers to be
                updated are in training mode when this hook is triggered.
            data_loader (iterable): it will produce data to be run by `model(data)`.
            num_iter (int): number of iterations used to compute the precise
                statistics.
        """
        self._logger = logging.getLogger(__name__)
        if len(get_bn_modules(model)) == 0:
            self._logger.info(
                "PreciseBN is disabled because model does not contain BN layers in training mode."
            )
            self._disabled = True
            return

        self._model = model
        self._data_loader = data_loader
        self._num_iter = num_iter
        self._disabled = False

        self._data_iter = None

    def after_epoch(self):
        next_epoch = self.trainer.epoch + 1
        is_final = next_epoch == self.trainer.max_epoch
        if is_final:
            self.update_stats()

    def update_stats(self):
        """
        Update the model with precise statistics. Users can manually call this method.
        """
        if self._disabled:
            return

        if self._data_iter is None:
            self._data_iter = iter(self._data_loader)

        def data_loader():
            for num_iter in itertools.count(1):
                if num_iter % 100 == 0:
                    self._logger.info(
                        "Running precise-BN ... {}/{} iterations.".format(num_iter, self._num_iter)
                    )
                # This way we can reuse the same iterator
                yield next(self._data_iter)

        with EventStorage():  # capture events in a new storage to discard them
            self._logger.info(
                "Running precise-BN for {} iterations...  ".format(self._num_iter)
                + "Note that this could produce different statistics every time."
            )
            update_bn_stats(self._model, data_loader(), self._num_iter)


class LayerFreeze(HookBase):
    def __init__(self, model, freeze_layers, freeze_iters):
        self._logger = logging.getLogger(__name__)
        if isinstance(model, DistributedDataParallel):
            model = model.module
        self.model = model

        self.freeze_layers = freeze_layers
        self.freeze_iters = freeze_iters

        self.is_frozen = False

    def before_step(self):
        # Freeze specific layers
        if self.trainer.iter < self.freeze_iters and not self.is_frozen:
            self.freeze_specific_layer()

        # Recover original layers status
        if self.trainer.iter >= self.freeze_iters and self.is_frozen:
            self.open_all_layer()

    def freeze_specific_layer(self):
        for layer in self.freeze_layers:
            if not hasattr(self.model, layer):
                self._logger.info(f'{layer} is not an attribute of the model, will skip this layer')

        for name, module in self.model.named_children():
            if name in self.freeze_layers:
                # Change BN in freeze layers to eval mode
                module.eval()

        self.is_frozen = True
        freeze_layers = ", ".join(self.freeze_layers)
        self._logger.info(f'Freeze layer group "{freeze_layers}" training for {self.freeze_iters:d} iterations')

    def open_all_layer(self):
        for name, module in self.model.named_children():
            if name in self.freeze_layers:
                module.train()

        self.is_frozen = False

        freeze_layers = ", ".join(self.freeze_layers)
        self._logger.info(f'Open layer group "{freeze_layers}" training')


class SWA(HookBase):
    def __init__(self, swa_start: int, swa_freq: int, swa_lr_factor: float, eta_min: float, lr_sched=False, ):
        self.swa_start = swa_start
        self.swa_freq = swa_freq
        self.swa_lr_factor = swa_lr_factor
        self.eta_min = eta_min
        self.lr_sched = lr_sched

    def before_step(self):
        is_swa = self.trainer.iter == self.swa_start
        if is_swa:
            # Wrapper optimizer with SWA
            self.trainer.optimizer = optim.SWA(self.trainer.optimizer, self.swa_freq, self.swa_lr_factor)
            self.trainer.optimizer.reset_lr_to_swa()

            if self.lr_sched:
                self.scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
                    optimizer=self.trainer.optimizer,
                    T_0=self.swa_freq,
                    eta_min=self.eta_min,
                )

    def after_step(self):
        next_iter = self.trainer.iter + 1

        # Use Cyclic learning rate scheduler
        if next_iter > self.swa_start and self.lr_sched:
            self.scheduler.step()

        is_final = next_iter == self.trainer.max_iter
        if is_final:
            self.trainer.optimizer.swap_swa_param()


In [25]:
# -*- coding: utf-8 -*-
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved

"""
This file contains components with some default boilerplate logic user may need
in training / testing. They will not work for everyone, but many users may find them useful.
The behavior of functions/classes in this file is subject to change,
since they are meant to represent the "common default behavior" people need in their projects.
"""

import argparse
import logging
import os
import sys
from collections import OrderedDict

import torch
from torch.nn.parallel import DistributedDataParallel

from fastreid.data import build_reid_test_loader, build_reid_train_loader
from fastreid.evaluation import (ReidEvaluator,
                                 inference_on_dataset, print_csv_format)
from fastreid.modeling.meta_arch import build_model
from fastreid.solver import build_lr_scheduler, build_optimizer
from fastreid.utils import comm
from fastreid.utils.checkpoint import Checkpointer
from fastreid.utils.collect_env import collect_env_info
from fastreid.utils.env import seed_all_rng
from fastreid.utils.events import CommonMetricPrinter, JSONWriter, TensorboardXWriter
from fastreid.utils.file_io import PathManager
from fastreid.utils.logger import setup_logger
# from . import hooks
# from .train_loop import TrainerBase, AMPTrainer, SimpleTrainer

__all__ = ["default_argument_parser", "default_setup", "DefaultPredictor", "DefaultTrainer"]


def default_argument_parser():
    """
    Create a parser with some common arguments used by fastreid users.
    Returns:
        argparse.ArgumentParser:
    """
    parser = argparse.ArgumentParser(description="fastreid Training")
    parser.add_argument("--config-file", default="", metavar="FILE", help="path to config file")
    parser.add_argument(
        "--resume",
        action="store_true",
        help="whether to attempt to resume from the checkpoint directory",
    )
    parser.add_argument("--eval-only", action="store_true", help="perform evaluation only")
    parser.add_argument("--num-gpus", type=int, default=1, help="number of gpus *per machine*")
    parser.add_argument("--num-machines", type=int, default=1, help="total number of machines")
    parser.add_argument(
        "--machine-rank", type=int, default=0, help="the rank of this machine (unique per machine)"
    )

    # PyTorch still may leave orphan processes in multi-gpu training.
    # Therefore we use a deterministic way to obtain port,
    # so that users are aware of orphan processes by seeing the port occupied.
    port = 2 ** 15 + 2 ** 14 + hash(os.getuid() if sys.platform != "win32" else 1) % 2 ** 14
    parser.add_argument("--dist-url", default="tcp://127.0.0.1:{}".format(port))
    parser.add_argument(
        "opts",
        help="Modify config options using the command-line",
        default=None,
        nargs=argparse.REMAINDER,
    )
    return parser


def default_setup(cfg, args):
    """
    Perform some basic common setups at the beginning of a job, including:
    1. Set up the detectron2 logger
    2. Log basic information about environment, cmdline arguments, and config
    3. Backup the config to the output directory
    Args:
        cfg (CfgNode): the full config to be used
        args (argparse.NameSpace): the command line arguments to be logged
    """
    output_dir = cfg.OUTPUT_DIR
    if comm.is_main_process() and output_dir:
        PathManager.mkdirs(output_dir)

    rank = comm.get_rank()
    # setup_logger(output_dir, distributed_rank=rank, name="fvcore")
    logger = setup_logger(output_dir, distributed_rank=rank)

    logger.info("Rank of current process: {}. World size: {}".format(rank, comm.get_world_size()))
    logger.info("Environment info:\n" + collect_env_info())

    logger.info("Command line arguments: " + str(args))
    if hasattr(args, "config_file") and args.config_file != "":
        logger.info(
            "Contents of args.config_file={}:\n{}".format(
                args.config_file, PathManager.open(args.config_file, "r").read()
            )
        )

    logger.info("Running with full config:\n{}".format(cfg))
    if comm.is_main_process() and output_dir:
        # Note: some of our scripts may expect the existence of
        # config.yaml in output directory
        path = os.path.join(output_dir, "config.yaml")
        with PathManager.open(path, "w") as f:
            f.write(cfg.dump())
        logger.info("Full config saved to {}".format(os.path.abspath(path)))

    # make sure each worker has a different, yet deterministic seed if specified
    seed_all_rng()

    # cudnn benchmark has large overhead. It shouldn't be used considering the small size of
    # typical validation set.
    if not (hasattr(args, "eval_only") and args.eval_only):
        torch.backends.cudnn.benchmark = cfg.CUDNN_BENCHMARK


class DefaultPredictor:
    """
    Create a simple end-to-end predictor with the given config.
    The predictor takes an BGR image, resizes it to the specified resolution,
    runs the model and produces a dict of predictions.
    This predictor takes care of model loading and input preprocessing for you.
    If you'd like to do anything more fancy, please refer to its source code
    as examples to build and use the model manually.
    Attributes:
    Examples:
    .. code-block:: python
        pred = DefaultPredictor(cfg)
        inputs = cv2.imread("input.jpg")
        outputs = pred(inputs)
    """

    def __init__(self, cfg):
        self.cfg = cfg.clone()  # cfg can be modified by model
        self.cfg.defrost()
        self.cfg.MODEL.BACKBONE.PRETRAIN = False
        self.model = build_model(self.cfg)
        self.model.eval()

        Checkpointer(self.model).load(cfg.MODEL.WEIGHTS)

    def __call__(self, image):
        """
        Args:
            image (torch.tensor): an image tensor of shape (B, C, H, W).
        Returns:
            predictions (torch.tensor): the output features of the model
        """
        inputs = {"images": image.to(self.model.device)}
        with torch.no_grad():  # https://github.com/sphinx-doc/sphinx/issues/4258
            predictions = self.model(inputs)
        return predictions.cpu()


class DefaultTrainer(TrainerBase):
    """
    A trainer with default training logic. Compared to `SimpleTrainer`, it
    contains the following logic in addition:
    1. Create model, optimizer, scheduler, dataloader from the given config.
    2. Load a checkpoint or `cfg.MODEL.WEIGHTS`, if exists.
    3. Register a few common hooks.
    It is created to simplify the **standard model training workflow** and reduce code boilerplate
    for users who only need the standard training workflow, with standard features.
    It means this class makes *many assumptions* about your training logic that
    may easily become invalid in a new research. In fact, any assumptions beyond those made in the
    :class:`SimpleTrainer` are too much for research.
    The code of this class has been annotated about restrictive assumptions it mades.
    When they do not work for you, you're encouraged to:
    1. Overwrite methods of this class, OR:
    2. Use :class:`SimpleTrainer`, which only does minimal SGD training and
       nothing else. You can then add your own hooks if needed. OR:
    3. Write your own training loop similar to `tools/plain_train_net.py`.
    Also note that the behavior of this class, like other functions/classes in
    this file, is not stable, since it is meant to represent the "common default behavior".
    It is only guaranteed to work well with the standard models and training workflow in fastreid.
    To obtain more stable behavior, write your own training logic with other public APIs.
    Attributes:
        scheduler:
        checkpointer:
        cfg (CfgNode):
    Examples:
    .. code-block:: python
        trainer = DefaultTrainer(cfg)
        trainer.resume_or_load()  # load last checkpoint or MODEL.WEIGHTS
        trainer.train()
    """

    def __init__(self, cfg):
        """
        Args:
            cfg (CfgNode):
        """
        super().__init__()
        logger = logging.getLogger("fastreid")
        if not logger.isEnabledFor(logging.INFO):  # setup_logger is not called for fastreid
            setup_logger()

        # Assume these objects must be constructed in this order.
        data_loader = self.build_train_loader(cfg)
        cfg = self.auto_scale_hyperparams(cfg, data_loader.dataset.num_classes)
        model = self.build_model(cfg)
        optimizer, param_wrapper = self.build_optimizer(cfg, model)

        # For training, wrap with DDP. But don't need this for inference.
        if comm.get_world_size() > 1:
            # ref to https://github.com/pytorch/pytorch/issues/22049 to set `find_unused_parameters=True`
            # for part of the parameters is not updated.
            model = DistributedDataParallel(
                model, device_ids=[comm.get_local_rank()], broadcast_buffers=False,
            )

        self._trainer = (AMPTrainer if cfg.SOLVER.AMP.ENABLED else SimpleTrainer)(
            model, data_loader, optimizer, param_wrapper
        )

        self.iters_per_epoch = len(data_loader.dataset) // cfg.SOLVER.IMS_PER_BATCH
        self.scheduler = self.build_lr_scheduler(cfg, optimizer, self.iters_per_epoch)

        # Assume no other objects need to be checkpointed.
        # We can later make it checkpoint the stateful hooks
        self.checkpointer = Checkpointer(
            # Assume you want to save checkpoints together with logs/statistics
            model,
            cfg.OUTPUT_DIR,
            save_to_disk=comm.is_main_process(),
            optimizer=optimizer,
            **self.scheduler,
        )

        self.start_epoch = 0
        self.max_epoch = cfg.SOLVER.MAX_EPOCH
        self.max_iter = self.max_epoch * self.iters_per_epoch
        self.warmup_iters = cfg.SOLVER.WARMUP_ITERS
        self.delay_epochs = cfg.SOLVER.DELAY_EPOCHS
        self.cfg = cfg

        self.register_hooks(self.build_hooks())

    def resume_or_load(self, resume=True):
        """
        If `resume==True` and `cfg.OUTPUT_DIR` contains the last checkpoint (defined by
        a `last_checkpoint` file), resume from the file. Resuming means loading all
        available states (eg. optimizer and scheduler) and update iteration counter
        from the checkpoint. ``cfg.MODEL.WEIGHTS`` will not be used.
        Otherwise, this is considered as an independent training. The method will load model
        weights from the file `cfg.MODEL.WEIGHTS` (but will not load other states) and start
        from iteration 0.
        Args:
            resume (bool): whether to do resume or not
        """
        # The checkpoint stores the training iteration that just finished, thus we start
        # at the next iteration (or iter zero if there's no checkpoint).
        checkpoint = self.checkpointer.resume_or_load(self.cfg.MODEL.WEIGHTS, resume=resume)

        if resume and self.checkpointer.has_checkpoint():
            self.start_epoch = checkpoint.get("epoch", -1) + 1
            # The checkpoint stores the training iteration that just finished, thus we start
            # at the next iteration (or iter zero if there's no checkpoint).

    def build_hooks(self):
        """
        Build a list of default hooks, including timing, evaluation,
        checkpointing, lr scheduling, precise BN, writing events.
        Returns:
            list[HookBase]:
        """
        logger = logging.getLogger(__name__)
        cfg = self.cfg.clone()
        cfg.defrost()
        cfg.DATALOADER.NUM_WORKERS = 0  # save some memory and time for PreciseBN
        cfg.DATASETS.NAMES = tuple([cfg.TEST.PRECISE_BN.DATASET])  # set dataset name for PreciseBN

        ret = [
            hooks.IterationTimer(),
            hooks.LRScheduler(self.optimizer, self.scheduler),
        ]

        if cfg.TEST.PRECISE_BN.ENABLED and hooks.get_bn_modules(self.model):
            logger.info("Prepare precise BN dataset")
            ret.append(hooks.PreciseBN(
                # Run at the same freq as (but before) evaluation.
                self.model,
                # Build a new data loader to not affect training
                self.build_train_loader(cfg),
                cfg.TEST.PRECISE_BN.NUM_ITER,
            ))

        if len(cfg.MODEL.FREEZE_LAYERS) > 0 and cfg.SOLVER.FREEZE_ITERS > 0:
            ret.append(hooks.LayerFreeze(
                self.model,
                cfg.MODEL.FREEZE_LAYERS,
                cfg.SOLVER.FREEZE_ITERS,
            ))

        # Do PreciseBN before checkpointer, because it updates the model and need to
        # be saved by checkpointer.
        # This is not always the best: if checkpointing has a different frequency,
        # some checkpoints may have more precise statistics than others.

        def test_and_save_results():
            self._last_eval_results = self.test(self.cfg, self.model)
            return self._last_eval_results

        # Do evaluation before checkpointer, because then if it fails,
        # we can use the saved checkpoint to debug.
        ret.append(hooks.EvalHook(cfg.TEST.EVAL_PERIOD, test_and_save_results))

        if comm.is_main_process():
            ret.append(hooks.PeriodicCheckpointer(self.checkpointer, cfg.SOLVER.CHECKPOINT_PERIOD))
            # run writers in the end, so that evaluation metrics are written
            ret.append(hooks.PeriodicWriter(self.build_writers(), 200))

        return ret

    def build_writers(self):
        """
        Build a list of writers to be used. By default it contains
        writers that write metrics to the screen,
        a json file, and a tensorboard event file respectively.
        If you'd like a different list of writers, you can overwrite it in
        your trainer.
        Returns:
            list[EventWriter]: a list of :class:`EventWriter` objects.
        It is now implemented by:
        .. code-block:: python
            return [
                CommonMetricPrinter(self.max_iter),
                JSONWriter(os.path.join(self.cfg.OUTPUT_DIR, "metrics.json")),
                TensorboardXWriter(self.cfg.OUTPUT_DIR),
            ]
        """
        # Assume the default print/log frequency.
        return [
            # It may not always print what you want to see, since it prints "common" metrics only.
            CommonMetricPrinter(self.max_iter),
            JSONWriter(os.path.join(self.cfg.OUTPUT_DIR, "metrics.json")),
            TensorboardXWriter(self.cfg.OUTPUT_DIR),
        ]

    def train(self):
        """
        Run training.
        Returns:
            OrderedDict of results, if evaluation is enabled. Otherwise None.
        """
        super().train(self.start_epoch, self.max_epoch, self.iters_per_epoch)
        if comm.is_main_process():
            assert hasattr(
                self, "_last_eval_results"
            ), "No evaluation results obtained during training!"
            return self._last_eval_results

    def run_step(self):
        self._trainer.iter = self.iter
        self._trainer.run_step()

    @classmethod
    def build_model(cls, cfg):
        """
        Returns:
            torch.nn.Module:
        It now calls :func:`fastreid.modeling.build_model`.
        Overwrite it if you'd like a different model.
        """
        model = build_model(cfg)
        logger = logging.getLogger(__name__)
        logger.info("Model:\n{}".format(model))
        return model

    @classmethod
    def build_optimizer(cls, cfg, model):
        """
        Returns:
            torch.optim.Optimizer:
        It now calls :func:`fastreid.solver.build_optimizer`.
        Overwrite it if you'd like a different optimizer.
        """
        return build_optimizer(cfg, model)

    @classmethod
    def build_lr_scheduler(cls, cfg, optimizer, iters_per_epoch):
        """
        It now calls :func:`fastreid.solver.build_lr_scheduler`.
        Overwrite it if you'd like a different scheduler.
        """
        return build_lr_scheduler(cfg, optimizer, iters_per_epoch)

    @classmethod
    def build_train_loader(cls, cfg):
        """
        Returns:
            iterable
        It now calls :func:`fastreid.data.build_reid_train_loader`.
        Overwrite it if you'd like a different data loader.
        """
        logger = logging.getLogger(__name__)
        logger.info("Prepare training set")
        return build_reid_train_loader(cfg, combineall=cfg.DATASETS.COMBINEALL)

    @classmethod
    def build_test_loader(cls, cfg, dataset_name):
        """
        Returns:
            iterable
        It now calls :func:`fastreid.data.build_reid_test_loader`.
        Overwrite it if you'd like a different data loader.
        """
        return build_reid_test_loader(cfg, dataset_name=dataset_name)

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_dir=None):
        data_loader, num_query = cls.build_test_loader(cfg, dataset_name)
        return data_loader, ReidEvaluator(cfg, num_query, output_dir)

    @classmethod
    def test(cls, cfg, model):
        """
        Args:
            cfg (CfgNode):
            model (nn.Module):
        Returns:
            dict: a dict of result metrics
        """
        logger = logging.getLogger(__name__)

        results = OrderedDict()
        for idx, dataset_name in enumerate(cfg.DATASETS.TESTS):
            logger.info("Prepare testing set")
            try:
                data_loader, evaluator = cls.build_evaluator(cfg, dataset_name)
            except NotImplementedError:
                logger.warn(
                    "No evaluator found. implement its `build_evaluator` method."
                )
                results[dataset_name] = {}
                continue
            results_i = inference_on_dataset(model, data_loader, evaluator, flip_test=cfg.TEST.FLIP.ENABLED)
            results[dataset_name] = results_i

            if comm.is_main_process():
                assert isinstance(
                    results, dict
                ), "Evaluator must return a dict on the main process. Got {} instead.".format(
                    results
                )
                logger.info("Evaluation results for {} in csv format:".format(dataset_name))
                results_i['dataset'] = dataset_name
                print_csv_format(results_i)

        if len(results) == 1:
            results = list(results.values())[0]

        return results

    @staticmethod
    def auto_scale_hyperparams(cfg, num_classes):
        r"""
        This is used for auto-computation actual training iterations,
        because some hyper-param, such as MAX_ITER, means training epochs rather than iters,
        so we need to convert specific hyper-param to training iterations.
        """
        cfg = cfg.clone()
        frozen = cfg.is_frozen()
        cfg.defrost()

        # If you don't hard-code the number of classes, it will compute the number automatically
        if cfg.MODEL.HEADS.NUM_CLASSES == 0:
            output_dir = cfg.OUTPUT_DIR
            cfg.MODEL.HEADS.NUM_CLASSES = num_classes
            logger = logging.getLogger(__name__)
            logger.info(f"Auto-scaling the num_classes={cfg.MODEL.HEADS.NUM_CLASSES}")

            # Update the saved config file to make the number of classes valid
            if comm.is_main_process() and output_dir:
                # Note: some of our scripts may expect the existence of
                # config.yaml in output directory
                path = os.path.join(output_dir, "config.yaml")
                with PathManager.open(path, "w") as f:
                    f.write(cfg.dump())

        if frozen: cfg.freeze()

        return cfg


# Access basic attributes from the underlying trainer
for _attr in ["model", "data_loader", "optimizer", "grad_scaler"]:
    setattr(DefaultTrainer, _attr, property(lambda self, x=_attr: getattr(self._trainer, x, None)))


In [26]:
from fastreid.modeling import build_model

In [27]:
# from torchvision import transforms
# from fastreid.modeling import build_model
# # Define transformation to tensor and normalization (based on your ReID model's requirements)
# transform = transforms.Compose([
#     transforms.ToPILImage(),
#     transforms.Resize((256, 128)),  # standard size for ReID models like FastReID
#     transforms.ToTensor(),
#     # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet stats
# ])

# import cv2 as cv
# import time
# import torch
# from ultralytics import YOLO

# # --- FastReID Setup ---
# from fastreid.config import get_cfg
# # from fastreid.engine import DefaultPredictor

# def setup_reid_predictor():
#     cfg = get_cfg()
#     cfg.merge_from_file("C:/CCTV/fast-reid/configs/Market1501/bagtricks_R50.yml")
#     cfg.MODEL.WEIGHTS = "C:/CCTV/market_bot_R50.pth" # The model you downloaded
#     cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
#     cfg.freeze()
#     return (cfg)

# # --- Initialize Models and Identity Management ---
# model = YOLO("yolov8n.pt") 
# reid_predictor = setup_reid_predictor()
# rmodel = build_model(reid_predictor)
# Checkpointer(rmodel).load(reid_predictor.MODEL.WEIGHTS)
# rmodel.eval()
# # This is our "memory" of who we've seen
# tracked_identities = {}
# next_permanent_id = 0
# similarity_threshold = 0.85

# ptime = 0

# # Use a basic tracker here; we will handle the ReID part ourselves
# for result in model.track(source="palace.mp4", stream=True, tracker="bytetrack.yaml", classes=[0]):
#     frame = result.orig_img
    
#     if result.boxes.id is not None:
#         boxes = result.boxes.xyxy.cpu().numpy()

#         for box in boxes:
#             x1, y1, x2, y2 = map(int, box)
            
#             # Crop the detected person
#             crop = frame[y1:y2, x1:x2]
#             if crop.size == 0: continue
#             tensor_crop = transform(crop).unsqueeze(0).to(reid_predictor.MODEL.DEVICE) 
#             # Extract visual features
#             rmodel.eval()
#             features = rmodel(tensor_crop)[0]
            
#             # --- Matching Logic ---
#             best_match_id = -1
#             max_similarity = -1

#             # Compare the new features to all known identities
#             for pid, stored_feature in tracked_identities.items():
#                 similarity = torch.nn.functional.cosine_similarity(features.unsqueeze(0), stored_feature.unsqueeze(0)).item()
#                 if similarity > max_similarity:
#                     max_similarity = similarity
#                     best_match_id = pid
            
#             final_id = -1
#             if max_similarity > similarity_threshold:
#                 final_id = best_match_id # It's a match!
#                 tracked_identities[final_id] = features # Update features
#             else:
#                 final_id = next_permanent_id # It's a new person
                
#                 tracked_identities[final_id] = features
#                 next_permanent_id += 1
#             print(len(tracked_identities),tracked_identities)

#             # --- Drawing Logic ---
#             label = f"Person ID: {final_id}"
#             cv.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
#             cv.putText(frame, label, (x1, y1 - 10), cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

#     # FPS Calculation
#     ctime = time.time()
#     if ptime != 0:
#         fps = 1 / (ctime - ptime)
#         cv.putText(frame, f"FPS: {int(fps)}", (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
#     ptime = ctime

#     cv.imshow("Manual ReID Tracking", frame)
#     if cv.waitKey(1) == ord('q'):
#         break

# cv.destroyAllWindows()

In [28]:
# from torchvision import transforms
# import cv2 as cv
# import time
# import torch
# from ultralytics import YOLO
# from fastreid.config import get_cfg


# # --- Transformations (You may not need this if using DefaultPredictor directly) ---
# # DefaultPredictor handles resizing and normalization internally.
# # This manual transform is good for custom models but might be redundant here.
# # transform = transforms.Compose([...])
# transform = transforms.Compose([
#     transforms.ToPILImage(),
#     transforms.Resize((256, 128)),  # Standard input size for Re-ID models
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])
# def setup_reid_predictor():
#     cfg = get_cfg()
#     cfg.merge_from_file("fast-reid/configs/Market1501/bagtricks_R50.yml")
#     cfg.MODEL.WEIGHTS = "market_bot_R50.pth"
#     cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
#     cfg.freeze()
#     return (cfg)

# # --- Initialize ---
# model = YOLO("yolov8n.pt") 
# reid_predictor = setup_reid_predictor()
# rmodel = build_model(reid_predictor)
# Checkpointer(rmodel).load(reid_predictor.MODEL.WEIGHTS)
# rmodel.eval()

# # Our "permanent memory" of visual features
# tracked_identities = {}
# next_permanent_id = 0

# # --- NEW: A map to link temporary tracker IDs to our permanent Re-IDs ---
# tracker_to_reid = {}

# similarity_threshold = 0.90 # Lowered slightly for more robust matching
# ptime = 0

# # Use YOLO's tracker to get stable temporary IDs
# for result in model.track(source="palace.mp4", stream=True, tracker="bytetrack.yaml",classes=[0]):
#     frame = result.orig_img
    
#     if result.boxes.id is not None:
#         boxes = result.boxes.xyxy.cpu().numpy()
#         tracker_ids = result.boxes.id.cpu().numpy().astype(int)

#         # --- MODIFIED LOGIC: Iterate over boxes AND their temporary tracker IDs ---
#         for box, tracker_id in zip(boxes, tracker_ids):
#             final_id = -1
            
#             # --- Check if we've already assigned a permanent ID to this track ---
#             if tracker_id in tracker_to_reid:
#                 final_id = tracker_to_reid[tracker_id]
#             else:
#                 # --- This is a NEW track, so we perform Re-ID ---
#                 x1, y1, x2, y2 = map(int, box)
#                 crop = frame[y1:y2, x1:x2]
#                 if crop.size == 0: continue
#                 tensor_crop = transform(crop).unsqueeze(0).to(reid_predictor.MODEL.DEVICE)
#                 # Extract visual features
#                 features = rmodel(tensor_crop)
#                 features = torch.nn.functional.normalize(features, p=2, dim=1)
#                 # Extract features
#                 # features = reid_predictor([crop])[0]
                
#                 # --- Matching Logic ---
#                 best_match_id = -1
#                 max_similarity = -1

#                 for pid, stored_feature in tracked_identities.items():
#                     similarity = torch.nn.functional.cosine_similarity(features, stored_feature).item()
#                     if similarity > max_similarity:
#                         max_similarity = similarity
#                         best_match_id = pid
                
#                 if max_similarity > similarity_threshold:
#                     # It's a match with a previously lost person
#                     final_id = best_match_id
#                 else:
#                     # It's a brand-new person
#                     final_id = next_permanent_id
#                     tracked_identities[final_id] = features
#                     next_permanent_id += 1
                
#                 # --- CRUCIAL: Link the temporary ID to our new/matched permanent ID ---
#                 tracker_to_reid[tracker_id] = final_id

#             # --- Drawing Logic (now uses the stable final_id) ---
#             print(len(tracked_identities),tracked_identities)
#             x1, y1, x2, y2 = map(int, box)
#             label = f"Person ID: {final_id}"
#             cv.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
#             cv.putText(frame, label, (x1, y1 - 10), cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

#     # FPS Calculation
#     ctime = time.time()
#     if ptime != 0:
#         fps = 1 / (ctime - ptime)
#         cv.putText(frame, f"FPS: {int(fps)}", (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
#     ptime = ctime

#     cv.imshow("Manual ReID Tracking", frame)
#     if cv.waitKey(1) == ord('q'):
#         break

# cv.destroyAllWindows()

In [29]:
# # --- IMPORTS ---
# import cv2 as cv
# import time
# import torch
# from torchvision import transforms
# from ultralytics import YOLO
# import torchvision.transforms.functional as F


# # Import what's needed from FastReID
# from fastreid.config import get_cfg
# from fastreid.modeling import build_model
# from fastreid.utils.checkpoint import Checkpointer


# # --- STEP 1: DEFINE THE IMAGE TRANSFORM ---
# # This was missing. It's essential to resize and normalize the images
# # exactly as the model expects.
# # <-- ADD THIS ENTIRE BLOCK
# transform = transforms.Compose([
#     transforms.ToPILImage(),
#     transforms.Resize((256, 128)),  # Standard input size for Re-ID models
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])


# def setup_reid_predictor():
#     """Sets up the configuration for the FastReID model."""
#     cfg = get_cfg()
#     cfg.merge_from_file("C:/CCTV/fast-reid/configs/Market1501/bagtricks_R50.yml")
#     # This path must be correct.
#     cfg.MODEL.WEIGHTS = "C:/CCTV/market_bot_R50.pth"
#     cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
#     cfg.freeze()
#     return cfg

# # --- INITIALIZE MODELS ---
# print("Initializing models...")
# # Object detection model
# model = YOLO("yolov8n.pt")

# # Re-identification model
# reid_cfg = setup_reid_predictor()
# rmodel = build_model(reid_cfg)
# Checkpointer(rmodel).load(reid_cfg.MODEL.WEIGHTS)
# rmodel.to(reid_cfg.MODEL.DEVICE)
# rmodel.eval()
# print("Models initialized successfully.")

# # --- TRACKING DATA STRUCTURES ---
# # Our "permanent memory" of visual features for each person
# tracked_identities = {}
# next_permanent_id = 0

# # A map to link temporary tracker IDs from YOLO to our permanent Re-IDs
# tracker_to_reid = {}

# # This threshold will now be meaningful. You might need to adjust it.
# similarity_threshold = 0.50
# ptime = 0

# # --- MAIN VIDEO PROCESSING LOOP ---
# # Use source=0 for webcam
# for result in model.track(source=0, stream=True, tracker="bytetrack.yaml", classes=[0]):
#     frame = result.orig_img

#     if result.boxes.id is None:
#         # If no persons are tracked in the frame, just show it
#         cv.imshow("Person Re-ID Tracking", frame)
#         if cv.waitKey(1) == ord('q'):
#             break
#         continue

#     boxes = result.boxes.xyxy.cpu().numpy()
#     tracker_ids = result.boxes.id.cpu().numpy().astype(int)

#     # --- Iterate over each tracked person ---
#     for box, tracker_id in zip(boxes, tracker_ids):
#         final_id = -1
        
#         # Check if we've already assigned a permanent ID to this track
#         if tracker_id in tracker_to_reid:
#             final_id = tracker_to_reid[tracker_id]
#         else:
#             # --- This is a NEW track, so we perform Re-ID ---
#             x1, y1, x2, y2 = map(int, box)
#             crop = frame[y1:y2, x1:x2]
#             if crop.size == 0: continue

#             # 1. Prepare the image using the defined transform
#             tensor_crop = transform(crop).unsqueeze(0).to(reid_cfg.MODEL.DEVICE)
#             F.to_pil_image(tensor_crop.squeeze(0).cpu()).save("check_input.jpg")
#             # 2. Extract features
#             features = rmodel(tensor_crop)
            
#             # --- STEP 2: NORMALIZE THE FEATURES ---
#             # This was missing. It makes the comparison fair and meaningful.
#             # <-- ADD THIS LINE
#             features = torch.nn.functional.normalize(features, p=2, dim=1)
            
#             # --- Matching Logic ---
#             best_match_id = -1
#             max_similarity = -1

#             for pid, stored_feature in tracked_identities.items():
#                 # Compare the new person's features with all known persons
#                 similarity = torch.nn.functional.cosine_similarity(features, stored_feature).item()
#                 if similarity > max_similarity:
#                     max_similarity = similarity
#                     best_match_id = pid
            
#             if max_similarity > similarity_threshold:
#                 # It's a match! Re-assign the permanent ID of the matched person.
#                 final_id = best_match_id
#             else:
#                 # It's a brand-new person. Assign a new permanent ID.
#                 final_id = next_permanent_id
#                 tracked_identities[final_id] = features  # Store the NORMALIZED features
#                 next_permanent_id += 1
            
#             # Link the temporary YOLO tracker ID to our permanent Re-ID
#             tracker_to_reid[tracker_id] = final_id
#             print(len(tracked_identities),tracked_identities)
#         # --- Drawing Logic ---
#         x1, y1, x2, y2 = map(int, box)
#         label = f"ID: {final_id}"
#         cv.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
#         cv.putText(frame, label, (x1, y1 - 10), cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

#     # --- FPS Calculation ---
#     ctime = time.time()
#     if ptime != 0:
#         fps = 1 / (ctime - ptime)
#         cv.putText(frame, f"FPS: {int(fps)}", (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
#     ptime = ctime

#     cv.imshow("Person Re-ID Tracking", frame)
#     if cv.waitKey(1) == ord('q'):
#         break

# cv.destroyAllWindows()

In [30]:
# # --- IMPORTS ---
# import cv2 as cv
# from torchvision import transforms
# import torchvision.transforms.functional as F
# import time
# import torch
# # We no longer need torchvision.transforms!
# # from torchvision import transforms
# from ultralytics import YOLO

# # Import what's needed from FastReID
# from fastreid.config import get_cfg
# # Use the high-level predictor instead of manual components
# transform = transforms.Compose([
#     transforms.ToPILImage(),
#     transforms.Resize((256, 128)),  # Standard input size for Re-ID models
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])


# def setup_reid_predictor():
#     """Sets up the configuration for the FastReID model."""
#     cfg = get_cfg()
#     cfg.merge_from_file("C:/CCTV/fast-reid/configs/Market1501/bagtricks_R50.yml")
#     # This path must be correct.
#     cfg.MODEL.WEIGHTS = "C:/CCTV/market_bot_R50.pth"
#     cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
#     cfg.freeze()
#     return cfg

# # --- INITIALIZE MODELS ---
# print("Initializing models...")
# # Object detection model
# model = YOLO("yolov8n.pt")

# # --- NEW: INITIALIZE THE RE-ID PREDICTOR ---
# # This one line replaces the manual build_model, Checkpointer, and rmodel.eval()
# reid_cfg = setup_reid_predictor()
# predictor = DefaultPredictor(reid_cfg)
# print("Models initialized successfully.")

# # --- TRACKING DATA STRUCTURES ---
# tracked_identities = {}
# next_permanent_id = 0
# tracker_to_reid = {}

# # Start with a reasonable threshold
# similarity_threshold = 0.95
# ptime = 0

# # --- MAIN VIDEO PROCESSING LOOP ---
# for result in model.track(source="palace.mp4", stream=True, tracker="bytetrack.yaml", classes=[0]):
#     frame = result.orig_img

#     if result.boxes.id is None:
#         cv.imshow("Person Re-ID Tracking", frame)
#         if cv.waitKey(1) == ord('q'): break
#         continue

#     boxes = result.boxes.xyxy.cpu().numpy()
#     tracker_ids = result.boxes.id.cpu().numpy().astype(int)

#     for box, tracker_id in zip(boxes, tracker_ids):
#         final_id = -1
        
#         if tracker_id in tracker_to_reid:
#             final_id = tracker_to_reid[tracker_id]
#         else:
#             # --- This is the new, corrected block ---
#             # This is a NEW track, so we perform Re-ID
#             x1, y1, x2, y2 = map(int, box)
#             crop = frame[y1:y2, x1:x2]
#             if crop.size == 0: continue

#             # 1. Convert NumPy crop to a PyTorch tensor with the correct shape (B, C, H, W)
#            # Apply the correct transform (resize + normalize)
#             processed_crop = transform(crop).unsqueeze(0).to(reid_cfg.MODEL.DEVICE)

#             with torch.no_grad():
#                 predictor.model.eval()
#                 features = predictor(processed_crop)
#                 # features = torch.nn.functional.normalize(features, p=2, dim=1)

            
#             # --- Matching Logic ---
#             best_match_id = -1
#             max_similarity = -1

#             for pid, stored_feature in tracked_identities.items():
#                 similarity = torch.nn.functional.cosine_similarity(features, stored_feature).item()
#                 if similarity > max_similarity:
#                     max_similarity = similarity
#                     best_match_id = pid
            
#             if max_similarity > similarity_threshold:
#                 final_id = best_match_id
#                 # Optional: Update the stored feature with the new one to adapt to appearance changes
#                 tracked_identities[final_id] = features
#             else:
#                 final_id = next_permanent_id
#                 tracked_identities[final_id] = features
#                 next_permanent_id += 1
#             print(len(tracked_identities),tracked_identities)
#             tracker_to_reid[tracker_id] = final_id
            
#         # --- Drawing Logic ---
#         x1, y1, x2, y2 = map(int, box)
#         label = f"ID: {final_id}"
#         cv.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
#         cv.putText(frame, label, (x1, y1 - 10), cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

#     # --- FPS Calculation ---
#     ctime = time.time()
#     if ptime != 0:
#         fps = 1 / (ctime - ptime)
#         cv.putText(frame, f"FPS: {int(fps)}", (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
#     ptime = ctime

#     cv.imshow("Person Re-ID Tracking", frame)
#     if cv.waitKey(1) == ord('q'):
#         break

# cv.destroyAllWindows()

In [31]:

# import torch
# import os
# from pathlib import Path

# # It is assumed that this script is run from a directory that contains
# # the cloned 'fast-reid' repository.
# # Add the fast-reid repository to the Python path.
# import sys
# sys.path.append('fast-reid')

# from fastreid.config import get_cfg
# from fastreid.modeling import build_model
# from fastreid.utils.checkpoint import Checkpointer
# import fastreid.modeling.heads 
# import fastreid.modeling.backbones
# def setup_cfg(config_file, weights_file):
#     """
#     Creates a FastReID config object and merges it with the specified
#     configuration file and weights.
#     """
#     # 1. Get a copy of the default configuration
#     cfg = get_cfg()

#     # 2. Add project-specific configurations if any
#     # (Not needed for this baseline model)

#     # 3. Merge from the specified YAML configuration file
#     # This file contains the model architecture and dataset specifics.
#     cfg.merge_from_file(config_file)

#     # 4. Set the path to the pre-trained model weights.
#     # This is a critical step for loading the desired model state.
#     cfg.MODEL.WEIGHTS = weights_file

#     # 5. Set the device to use for inference.
#     # Use 'cuda' for GPU or 'cpu' for CPU.
#     cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
#     cfg.MODEL.HEADS.NUM_CLASSES = 751
#     # 6. Freeze the configuration.
#     # This makes the config object immutable and prevents accidental changes.
#     cfg.freeze()
    
#     return cfg

# def main():
#     """
#     Main function to load the model and print its architecture.
#     """
#     # --- Configuration ---
#     # Define paths relative to the project structure.
#     project_root = Path('.')
#     fastreid_repo_path = project_root / 'fast-reid'
    
#     # Path to the configuration YAML file.
#     config_file_path = fastreid_repo_path / 'configs/Market1501/bagtricks_R50.yml'
    
#     # Path to the pre-trained weights file.
#     weights_file_path = project_root / 'models/market_bot_R50.pth'

#     # Verify that the necessary files exist.
#     if not config_file_path.is_file():
#         print(f"Error: Config file not found at {config_file_path}")
#         return
#     if not weights_file_path.is_file():
#         print(f"Error: Weights file not found at {weights_file_path}")
#         print("Please download 'market_bot_R50.pth' and place it in the 'models/' directory.")
#         return

#     # --- Model Loading ---
#     # 1. Setup the configuration object.
#     cfg = setup_cfg(str(config_file_path), str(weights_file_path))
#     print(f"Configuration loaded successfully. Using device: {cfg.MODEL.DEVICE}")

#     # 2. Build the model.
#     # The `build_model` function reads the configuration and constructs the
#     # corresponding model architecture. It also automatically loads the weights
#     # specified in `cfg.MODEL.WEIGHTS`.
#     model = build_model(cfg)
#     print("Model built successfully.")

#     # The Checkpointer is used internally by `build_model` but can also be
#     # used explicitly if needed. For this use case, `build_model` handles it.
#     # Checkpointer(model).load(cfg.MODEL.WEIGHTS) # This is done inside build_model

#     # 3. Set the model to evaluation mode.
#     # This is crucial for consistent results during inference as it disables
#     # layers like Dropout and uses running statistics for Batch Normalization.
#     model.eval()
#     print("Model set to evaluation mode.")

#     # --- Robust Weight Verification ---
#    # --- Manual Weight Correction and Loading ---
#     print("\n--- Manually Correcting and Loading Weights ---")
    
#     device = torch.device(cfg.MODEL.DEVICE)
#     weights_from_file = torch.load(weights_file_path, map_location=device)
    
#     # Check if weights are nested under a 'model' key
#     if 'model' in weights_from_file:
#         weights_from_file = weights_from_file['model']

#     # Create a new dictionary to hold the corrected weights
#     from collections import OrderedDict
#     corrected_weights = OrderedDict()
    
#     # Iterate through the keys from the file and rename them
#     for k, v in weights_from_file.items():
#         # --- THIS IS THE FIX ---
#         if k in ['pixel_mean', 'pixel_std']:
#             continue
#         if k.startswith('module.'):
#              # Remove 'module.' prefix if it exists
#             new_key = k[7:]
#         else:
#             new_key = k
        
#         # Rename the head layers
#         if new_key == 'heads.classifier.weight':
#             new_key = 'heads.weight'
#         elif 'heads.bnneck.' in new_key:
#             new_key = new_key.replace('heads.bnneck.', 'heads.bottleneck.0.')
        
#         corrected_weights[new_key] = v
#         # --------------------

#     # Load the corrected weights into the model.
#     # We use strict=False to ignore keys that are in the file but not needed by the model
#     # (like pixel_mean, pixel_std).
#     incompatible = model.load_state_dict(corrected_weights, strict=False)
    
#     if not incompatible.missing_keys and not incompatible.unexpected_keys:
#         print("✅ Success! All model weights were loaded correctly after manual correction.")
#     else:
#         print("❌ Error! Still some mismatches after correction.")
#         print(f"Missing keys in model: {incompatible.missing_keys}")
#         print(f"Unexpected keys in file: {incompatible.unexpected_keys}")

#     print("--------------------------------\n")

#     # --- Verification ---
#     # ... (the rest of the script, like printing the model architecture) ...

#     # --- Verification ---
#     # Print the model architecture to verify it has been loaded correctly.
#     # You should see the ResNet-50 blocks, the pooling layer, and the BNNeck.
#     print("\n--- Model Architecture ---")
#     print(model)
#     print("--------------------------\n")
    
#     print("FastReID model is ready for feature extraction.")
    
#     # The 'model' object is now ready to be used for inference.
#     return model

# if __name__ == '__main__':
#     loaded_model = main()

In [5]:
torch.nn.functional.cosine_similarity(loaded_model(transform(cv.imread("C:/CCTV/images/1.jpg"))), loaded_model(transform(cv.imread("C:/CCTV/images/2.jpg")))).item()

0.4945070147514343

In [33]:
# import cv2 as cv
# import torch
# import torchvision.transforms as T
# from PIL import Image 
# from torchvision import transforms
# from fastreid.modeling import build_model
# # Define transformation to tensor and normalization (based on your ReID model's requirements)
# # transform = transforms.Compose([
# #     transforms.ToPILImage(),
# #     transforms.Resize((256, 128)),  # standard size for ReID models like FastReID
# #     transforms.ToTensor(),
# #     # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet stats
# # ])

# import cv2 as cv
# import time
# import torch
# from ultralytics import YOLO


# tracked_identities = {}
# next_permanent_id = 0
# similarity_threshold = 0.7

# ptime = 0

# # Use a basic tracker here; we will handle the ReID part ourselves
# for result in model.track(source="palace.mp4", stream=True, tracker="bytetrack.yaml", classes=[0]):
#     frame = result.orig_img
    
#     if result.boxes.id is not None:
#         boxes = result.boxes.xyxy.cpu().numpy()

#         for box in boxes:
#             x1, y1, x2, y2 = map(int, box)
            
#             # Crop the detected person
#             crop = frame[y1:y2, x1:x2]
#             if crop.size == 0: continue
#             tensor_crop = transform(crop) 
#             # Extract visual features
            
#             features = loaded_model(tensor_crop)
            
#             # --- Matching Logic ---
#             best_match_id = -1
#             max_similarity = -1

#             # Compare the new features to all known identities
#             for pid, stored_feature in tracked_identities.items():
#                 similarity = torch.nn.functional.cosine_similarity(features, stored_feature).item()
#                 if similarity > max_similarity:
#                     max_similarity = similarity
#                     best_match_id = pid
            
#             final_id = -1
#             if max_similarity > similarity_threshold:
#                 final_id = best_match_id # It's a match!
#                 tracked_identities[final_id] = features # Update features
#             else:
#                 final_id = next_permanent_id # It's a new person
                
#                 tracked_identities[final_id] = features
#                 next_permanent_id += 1
#             print(len(tracked_identities),tracked_identities)

#             # --- Drawing Logic ---
#             label = f"Person ID: {final_id}"
#             cv.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
#             cv.putText(frame, label, (x1, y1 - 10), cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

#     # FPS Calculation
#     ctime = time.time()
#     if ptime != 0:
#         fps = 1 / (ctime - ptime)
#         cv.putText(frame, f"FPS: {int(fps)}", (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
#     ptime = ctime

#     cv.imshow("Manual ReID Tracking", frame)
#     if cv.waitKey(1) == ord('q'):
#         break

# cv.destroyAllWindows()

In [4]:
# --- IMPORTS ---
import cv2 as cv
from torchvision import transforms
import torchvision.transforms.functional as F
import time
import torch
# We no longer need torchvision.transforms!
# from torchvision import transforms
from ultralytics import YOLO

# Import what's needed from FastReID
from fastreid.config import get_cfg
# # Use the high-level predictor instead of manual components
# transform = transforms.Compose([
#     transforms.ToPILImage(),
#     transforms.Resize((256, 128)),  # Standard input size for Re-ID models
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])


# --- TRACKING DATA STRUCTURES ---
tracked_identities = {}
next_permanent_id = 0
tracker_to_reid = {}

# Start with a reasonable threshold
similarity_threshold = 0.9
ptime = 0

# --- MAIN VIDEO PROCESSING LOOP ---
for result in model3.track(source="palace.mp4", stream=True, tracker="bytetrack.yaml",classes=[0]):
    frame = result.orig_img

    if result.boxes.id is None:
        cv.imshow("Person Re-ID Tracking", frame)
        if cv.waitKey(1) == ord('q'): break
        continue

    boxes = result.boxes.xyxy.cpu().numpy()
    tracker_ids = result.boxes.id.cpu().numpy().astype(int)

    for box, tracker_id in zip(boxes, tracker_ids):
        final_id = -1
        
        if tracker_id in tracker_to_reid:
            final_id = tracker_to_reid[tracker_id]
        else:
            # --- This is the new, corrected block ---
            # This is a NEW track, so we perform Re-ID
            x1, y1, x2, y2 = map(int, box)
            crop = frame[y1:y2, x1:x2]
            if crop.size == 0: continue

            # 1. Convert NumPy crop to a PyTorch tensor with the correct shape (B, C, H, W)
           # Apply the correct transform (resize + normalize)
            processed_crop = transform(crop)

            with torch.no_grad():
                
                features = loaded_model(processed_crop)
                # features = torch.nn.functional.normalize(features, p=2, dim=1)

            
            # --- Matching Logic ---
            best_match_id = -1
            max_similarity = -1

            for pid, stored_feature in tracked_identities.items():
                similarity = torch.nn.functional.cosine_similarity(features, stored_feature).item()
                if similarity > max_similarity:
                    max_similarity = similarity
                    best_match_id = pid
            
            if max_similarity > similarity_threshold:
                final_id = best_match_id
                # Optional: Update the stored feature with the new one to adapt to appearance changes
                tracked_identities[final_id] = (tracked_identities[final_id]+features)/2
            else:
                final_id = next_permanent_id
                tracked_identities[final_id] = features
                next_permanent_id += 1
            print(len(tracked_identities),tracked_identities)
            tracker_to_reid[tracker_id] = final_id
            
        # --- Drawing Logic ---
        x1, y1, x2, y2 = map(int, box)
        label = f"ID: {final_id}"
        cv.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv.putText(frame, label, (x1, y1 - 10), cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # --- FPS Calculation ---
    ctime = time.time()
    if ptime != 0:
        fps = 1 / (ctime - ptime)
        cv.putText(frame, f"FPS: {int(fps)}", (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    ptime = ctime

    cv.imshow("Person Re-ID Tracking", frame)
    if cv.waitKey(1) == ord('q'):
        break

cv.destroyAllWindows()


video 1/1 (frame 1/329) c:\CCTV\palace.mp4: 384x640 10 persons, 36.3ms
1 {0: tensor([[-0.5008,  0.0561,  0.3553,  ..., -0.1541,  0.0792, -0.1903]], device='cuda:0')}
2 {0: tensor([[-0.5008,  0.0561,  0.3553,  ..., -0.1541,  0.0792, -0.1903]], device='cuda:0'), 1: tensor([[-1.0304, -1.0433,  0.2717,  ..., -0.1313,  1.2633, -0.0835]], device='cuda:0')}
3 {0: tensor([[-0.5008,  0.0561,  0.3553,  ..., -0.1541,  0.0792, -0.1903]], device='cuda:0'), 1: tensor([[-1.0304, -1.0433,  0.2717,  ..., -0.1313,  1.2633, -0.0835]], device='cuda:0'), 2: tensor([[-0.2401, -0.9490,  0.0121,  ..., -0.2927,  0.3419,  0.3385]], device='cuda:0')}
4 {0: tensor([[-0.5008,  0.0561,  0.3553,  ..., -0.1541,  0.0792, -0.1903]], device='cuda:0'), 1: tensor([[-1.0304, -1.0433,  0.2717,  ..., -0.1313,  1.2633, -0.0835]], device='cuda:0'), 2: tensor([[-0.2401, -0.9490,  0.0121,  ..., -0.2927,  0.3419,  0.3385]], device='cuda:0'), 3: tensor([[-0.6003, -0.2303,  0.2656,  ..., -0.0382,  0.1661, -0.7054]], device='cuda:0

In [18]:
tracked_identities

defaultdict(<function __main__.<lambda>()>,
            {0: deque([tensor([-1.0033, -0.9493,  0.3245,  ..., -0.1539,  1.0188, -0.0689], device='cuda:0'),
                    tensor([-0.9675, -0.8747,  0.6253,  ..., -0.0103,  0.9891, -0.1327], device='cuda:0'),
                    tensor([-0.7900, -1.0234,  0.5660,  ..., -0.1318,  1.3328, -0.1080], device='cuda:0'),
                    tensor([-0.7561, -0.9820,  0.2295,  ..., -0.2123,  0.3549, -0.0228], device='cuda:0'),
                    tensor([-0.6940, -1.0054,  0.0840,  ..., -0.1727,  1.2262,  0.4132], device='cuda:0'),
                    tensor([-0.9026, -0.9291,  0.3671,  ..., -0.1831,  0.9804, -0.0474], device='cuda:0'),
                    tensor([-0.8541, -0.9756,  0.5649,  ..., -0.1509,  0.7687,  0.3353], device='cuda:0'),
                    tensor([-0.6701, -0.9548,  0.4814,  ..., -0.0406,  1.5448,  0.0554], device='cuda:0'),
                    tensor([-0.6420, -0.9864,  0.5883,  ..., -0.0242,  1.7718, -0.0307], device='

In [22]:
torch.nn.functional.cosine_similarity(tracked_identities[0][0], tracked_identities[0])

TypeError: cosine_similarity(): argument 'x2' (position 2) must be Tensor, not collections.deque

In [17]:
import cv2 as cv
import torch
import time
from collections import deque, defaultdict
from ultralytics import YOLO
from torchvision import transforms

# --- Preprocessing for ReID ---


# --- Parameters ---
SIMILARITY_THRESHOLD = 0.92     # ReID cosine similarity threshold
FEATURE_HISTORY = 30           # Keep last N embeddings per ID
CONFIRM_FRAMES = 8             # Frames before promoting a new ID
MIN_BOX_AREA = 800              # Ignore tiny detections
pending_tracks = defaultdict(lambda: {"count": 0, "feature": None})

# --- Data structures ---
tracked_identities = defaultdict(lambda: deque(maxlen=FEATURE_HISTORY))  # Permanent IDs -> deque of features
tracker_to_reid = {}    # ByteTrack ID -> permanent ID
next_permanent_id = 0

# --- Load YOLO model ---
# model3 = YOLO("yolov8n.pt")  # Replace with your trained weights

# --- Load FastReID model ---


# --- Function: Cosine similarity between new feature and stored ones ---
# def best_match(feature, stored_features):
#     best_id = -1
#     best_sim = -1
#     for pid, feats in stored_features.items():
#         feats_tensor = torch.stack(list(feats))
#         sims = torch.nn.functional.cosine_similarity(feature, feats_tensor)
#         max_sim = sims.max().item()
        
#         if max_sim > best_sim:
#             best_sim = max_sim
#             best_id = pid
#     return best_id, best_sim
def best_match(feature, stored_features):
    best_id = -1
    best_score = -1

    for pid, feats in stored_features.items():
        feats_tensor = torch.stack(list(feats))
        
        # Max similarity
        max_sim = torch.nn.functional.cosine_similarity(feature, feats_tensor).max().item()
        
        # Average similarity
        avg_feat = feats_tensor.mean(dim=0, keepdim=True)
        avg_sim = torch.nn.functional.cosine_similarity(feature, avg_feat).item()
        
        # Weighted score
        final_score = 0.85 * max_sim + 0.15 * avg_sim

        if final_score > best_score:
            best_score = final_score
            best_id = pid

    return best_id, best_score

# --- Function: Preprocess crop for ReID ---
def preprocess_crop(frame, box):
    x1, y1, x2, y2 = map(int, box)
    crop = frame[y1:y2, x1:x2]
    if crop.size == 0 or (x2-x1)*(y2-y1) < MIN_BOX_AREA:
        return None
    return transform(crop)

# --- Main loop ---
ptime = 0
for result in model3.track(source=0, stream=True, tracker="bytetrack.yaml", classes=[0]):
    frame = result.orig_img

    if result.boxes.id is None:
        cv.imshow("Person Re-ID Tracking", frame)
        if cv.waitKey(1) == ord('q'):
            break
        continue

    boxes = result.boxes.xyxy.cpu().numpy()
    tracker_ids = result.boxes.id.cpu().numpy().astype(int)

    crops_to_match = []
    id_mapping = []

    # Step 1: Collect crops for unmatched tracker IDs
    for box, tracker_id in zip(boxes, tracker_ids):
        if tracker_id not in tracker_to_reid:
            crop_tensor = preprocess_crop(frame, box)
            if crop_tensor is not None:
                crops_to_match.append(crop_tensor)
                id_mapping.append(tracker_id)

    # Step 2: Batch ReID for new/unmatched tracker IDs
    if crops_to_match:
        batch_tensor = torch.cat(crops_to_match, dim=0)
        with torch.no_grad():
            features_batch = loaded_model(batch_tensor)

        for tracker_id, feature in zip(id_mapping, features_batch):
            best_id, sim = best_match(feature, tracked_identities)

            if sim > SIMILARITY_THRESHOLD:
                # Match found
                tracker_to_reid[tracker_id] = best_id
                tracked_identities[best_id].append(feature)
            else:
                # New track confirmation system
                pending_tracks[tracker_id]["count"] += 1
                if pending_tracks[tracker_id]["count"] >= CONFIRM_FRAMES:
                    global next_permanent_id
                    new_id = next_permanent_id
                    next_permanent_id += 1
                    tracker_to_reid[tracker_id] = new_id
                    tracked_identities[new_id].append(feature)
                    del pending_tracks[tracker_id]
                else:
                    pending_tracks[tracker_id]["feature"] = feature

    # Step 3: Drawing
    for box, tracker_id in zip(boxes, tracker_ids):
        x1, y1, x2, y2 = map(int, box)
        if tracker_id in tracker_to_reid:
            final_id = tracker_to_reid[tracker_id]
        else:
            final_id = -1  # Pending

        label = f"ID: {final_id}" if final_id != -1 else "Pending"
        cv.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv.putText(frame, label, (x1, y1 - 10),
                   cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # Step 4: FPS counter
    ctime = time.time()
    if ptime != 0:
        fps = 1 / (ctime - ptime)
        cv.putText(frame, f"FPS: {int(fps)}", (10, 30),
                   cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    ptime = ctime

    # Step 5: Show
    cv.imshow("Person Re-ID Tracking", frame)
    if cv.waitKey(1) == ord('q'):
        break

cv.destroyAllWindows()



1/1: 0... Success  (inf frames of shape 640x480 at 30.00 FPS)

0: 480x640 1 person, 1285.7ms
0: 480x640 1 person, 8.2ms
0: 480x640 1 person, 9.9ms
0: 480x640 1 person, 11.2ms
0: 480x640 1 person, 7.1ms
0: 480x640 1 person, 9.1ms
0: 480x640 1 person, 6.8ms
0: 480x640 1 person, 6.4ms
0: 480x640 1 person, 7.6ms
0: 480x640 1 person, 7.0ms
0: 480x640 1 person, 7.2ms
0: 480x640 1 person, 8.1ms
0: 480x640 1 person, 6.6ms
0: 480x640 1 person, 6.6ms
0: 480x640 1 person, 7.2ms
0: 480x640 1 person, 9.4ms
0: 480x640 1 person, 6.8ms
0: 480x640 1 person, 7.2ms
0: 480x640 1 person, 6.4ms
0: 480x640 1 person, 8.7ms
0: 480x640 1 person, 7.1ms
0: 480x640 1 person, 6.6ms
0: 480x640 1 person, 9.3ms
0: 480x640 1 person, 8.2ms
0: 480x640 1 person, 11.4ms
0: 480x640 1 person, 7.7ms
0: 480x640 1 person, 7.6ms
0: 480x640 1 person, 7.6ms
0: 480x640 1 person, 7.1ms
0: 480x640 1 person, 7.4ms
0: 480x640 1 person, 7.6ms
0: 480x640 1 person, 7.3ms
0: 480x640 1 person, 7.4ms
0: 480x640 1 person, 8.8ms
0: 480x640 1 

In [ ]:
import cv2 as cv
import torch
import time
import csv
import os
from collections import deque, defaultdict
from ultralytics import YOLO
from torchvision import transforms

# --- Parameters ---
SIMILARITY_THRESHOLD = 0.92
FEATURE_HISTORY = 30
CONFIRM_FRAMES = 8
MIN_BOX_AREA = 800
REAPPEAR_THRESHOLD = 2.0  # seconds

# --- Data structures ---
pending_tracks = defaultdict(lambda: {"count": 0, "feature": None})
tracked_identities = defaultdict(lambda: deque(maxlen=FEATURE_HISTORY))
tracker_to_reid = {}
next_permanent_id = 0

# --- Appearance/reappearance logging structures ---
last_seen_time = {}  # permanent_id -> last timestamp
appearance_log = defaultdict(list)  # permanent_id -> [(event_type, timestamp)]

# --- CSV logging setup ---
csv_file_path = os.path.abspath("person_appearance_log.csv")
write_header = not os.path.exists(csv_file_path)

csv_file = open(csv_file_path, mode="a", newline="")
csv_writer = csv.writer(csv_file)

if write_header:
    csv_writer.writerow(["PersonID", "EventType", "Time_HHMMSS"])
    csv_file.flush()

# --- Best match function ---
def best_match(feature, stored_features):
    best_id = -1
    best_score = -1
    for pid, feats in stored_features.items():
        feats_tensor = torch.stack(list(feats))
        max_sim = torch.nn.functional.cosine_similarity(feature, feats_tensor).max().item()
        avg_feat = feats_tensor.mean(dim=0, keepdim=True)
        avg_sim = torch.nn.functional.cosine_similarity(feature, avg_feat).item()
        final_score = 0.85 * max_sim + 0.15 * avg_sim
        if final_score > best_score:
            best_score = final_score
            best_id = pid
    return best_id, best_score

# --- Crop preprocessing ---
def preprocess_crop(frame, box):
    x1, y1, x2, y2 = map(int, box)
    crop = frame[y1:y2, x1:x2]
    if crop.size == 0 or (x2-x1)*(y2-y1) < MIN_BOX_AREA:
        return None
    return transform(crop)  # Make sure transform is defined

# --- Main loop ---
ptime = 0
for result in model3.track(source="palace.mp4", stream=True, tracker="bytetrack.yaml", classes=[0]):
    frame = result.orig_img
    current_time = time.time()

    if result.boxes.id is None:
        cv.imshow("Person Re-ID Tracking", frame)
        if cv.waitKey(1) == ord('q'):
            break
        continue

    boxes = result.boxes.xyxy.cpu().numpy()
    tracker_ids = result.boxes.id.cpu().numpy().astype(int)

    crops_to_match = []
    id_mapping = []

    # Step 1: Collect crops for unmatched tracker IDs
    for box, tracker_id in zip(boxes, tracker_ids):
        if tracker_id not in tracker_to_reid:
            crop_tensor = preprocess_crop(frame, box)
            if crop_tensor is not None:
                crops_to_match.append(crop_tensor)
                id_mapping.append(tracker_id)

    # Step 2: Batch ReID for new/unmatched tracker IDs
    if crops_to_match:
        batch_tensor = torch.cat(crops_to_match, dim=0)
        with torch.no_grad():
            features_batch = loaded_model(batch_tensor)
        for tracker_id, feature in zip(id_mapping, features_batch):
            best_id, sim = best_match(feature, tracked_identities)
            if sim > SIMILARITY_THRESHOLD:
                tracker_to_reid[tracker_id] = best_id
                tracked_identities[best_id].append(feature)
            else:
                pending_tracks[tracker_id]["count"] += 1
                if pending_tracks[tracker_id]["count"] >= CONFIRM_FRAMES:
                    global next_permanent_id
                    new_id = next_permanent_id
                    next_permanent_id += 1
                    tracker_to_reid[tracker_id] = new_id
                    tracked_identities[new_id].append(feature)
                    del pending_tracks[tracker_id]
                else:
                    pending_tracks[tracker_id]["feature"] = feature

    # Step 3: Drawing + CSV logging
    for box, tracker_id in zip(boxes, tracker_ids):
        x1, y1, x2, y2 = map(int, box)
        if tracker_id in tracker_to_reid:
            final_id = tracker_to_reid[tracker_id]

            # --- Detect appearance/reappearance ---
            if final_id not in last_seen_time:
                appearance_log[final_id].append(("appearance"))
                print(f"[{time.strftime('%H:%M:%S', time.localtime(current_time))}] Person {final_id} appeared.")
                csv_writer.writerow([final_id, "appearance",
                                     time.strftime("%H:%M:%S", time.localtime(current_time))
                                     ])
                csv_file.flush()

            else:
                gap = current_time - last_seen_time[final_id]
                if gap > REAPPEAR_THRESHOLD:
                    appearance_log[final_id].append(("reappearance"))
                    print(f"[{time.strftime('%H:%M:%S', time.localtime(current_time))}] Person {final_id} reappeared after {gap:.1f} sec.")
                    csv_writer.writerow([final_id, "reappearance",
                                         time.strftime("%H:%M:%S", time.localtime(current_time))
                                         ])
                    csv_file.flush()

            last_seen_time[final_id] = current_time
        else:
            final_id = -1  # Pending

        label = f"ID: {final_id}" if final_id != -1 else "Pending"
        cv.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv.putText(frame, label, (x1, y1 - 10),
                   cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # Step 4: FPS counter
    ctime = time.time()
    if ptime != 0:
        fps = 1 / (ctime - ptime)
        cv.putText(frame, f"FPS: {int(fps)}", (10, 30),
                   cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    ptime = ctime

    # Step 5: Show
    cv.imshow("Person Re-ID Tracking", frame)
    if cv.waitKey(1) == ord('q'):
        break

# --- Cleanup ---
csv_file.close()
cv.destroyAllWindows()



video 1/1 (frame 1/329) c:\CCTV\palace.mp4: 384x640 10 persons, 206.4ms
video 1/1 (frame 2/329) c:\CCTV\palace.mp4: 384x640 10 persons, 10.9ms
video 1/1 (frame 3/329) c:\CCTV\palace.mp4: 384x640 10 persons, 8.4ms
video 1/1 (frame 4/329) c:\CCTV\palace.mp4: 384x640 10 persons, 10.1ms
video 1/1 (frame 5/329) c:\CCTV\palace.mp4: 384x640 10 persons, 8.3ms
video 1/1 (frame 6/329) c:\CCTV\palace.mp4: 384x640 10 persons, 6.4ms
video 1/1 (frame 7/329) c:\CCTV\palace.mp4: 384x640 10 persons, 6.9ms
video 1/1 (frame 8/329) c:\CCTV\palace.mp4: 384x640 10 persons, 6.8ms
[01:55:56] Person 0 appeared.
[01:55:56] Person 1 appeared.
[01:55:56] Person 2 appeared.
[01:55:56] Person 3 appeared.
[01:55:56] Person 4 appeared.
[01:55:56] Person 5 appeared.
[01:55:56] Person 6 appeared.
[01:55:56] Person 7 appeared.
[01:55:56] Person 8 appeared.
[01:55:56] Person 9 appeared.
video 1/1 (frame 9/329) c:\CCTV\palace.mp4: 384x640 10 persons, 7.5ms
video 1/1 (frame 10/329) c:\CCTV\palace.mp4: 384x640 12 persons, 